In [1]:
import pandas as pd
pd.set_option('display.max_rows', 60)
pd.set_option('display.max_columns', None) 
import numpy as np
import re
from collections import defaultdict

In [2]:
yahoo_players = pd.read_csv('Raw/players_yahooleague_raw_14to25.csv')
yahoo_players.head()
yahoo_players = yahoo_players.drop(columns='Unnamed: 0')
yahoo_players.head()

master_ids = pd.read_csv('Raw/player_keys_raw.csv')
master_ids.head()
master_ids = master_ids.drop(columns='Unnamed: 0')
master_ids.head()

master_raw = master_ids.copy()

In [3]:
master_raw['position'].value_counts()

position
WR    2038
LB    1779
RB    1587
CB    1268
S     1083
DT     969
DE     957
TE     948
QB     666
PK     276
PN     215
DB     146
DL     136
OT      53
XX      16
C        6
T        1
?        1
Name: count, dtype: int64

In [4]:
master_raw[master_raw['name'] == 'Peyton Hillis']

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
6857,9272,a34a5944-33c4-4965-b404-6772f4b7f1ab,9506.0,00-0026367,4541.0,660.0,1980.0,11461.0,900035.0,NaN,523458.0,HillPe00,peyton-hillis-1,5866.0,4965.0,NaN,9004.0,267714.0,11747.0,NaN,Peyton Hillis,peyton hillis,RB,FA,1986-01-21,35.3,2008.0,7.0,20.0,227.0,thepeytonhillis,74.0,250.0,Arkansas,2025


In [5]:
master_raw = master_raw.loc[master_raw['position'].isin(['QB', 'RB', 'TE', 'WR', 'PK', 'XX'])]
master_raw[master_raw['name'] == 'Jacoby Jones']

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
7043,8772,ce4d7d70-307f-4093-bf54-432b8b405eb4,9253.0,00-0025460,3689.0,203.0,2495651.0,10517.0,8327.0,4674.0,1220258.0,JoneJa03,NaN,5229.0,4239.0,NaN,8327.0,329337.0,5100.0,NaN,Jacoby Jones,jacoby jones,WR,FA,1984-07-11,36.8,2007.0,3.0,9.0,73.0,TheRealJacoby12,76.0,215.0,Lane,2025


In [6]:
# Create name cleaning funciton
def clean_name(name):
    if pd.isna(name) or name is None:
        return name
    name = re.sub(r'\s+(Jr\.?|Sr\.?|III|II|IV|V)(\s|$|,)', '', str(name), flags=re.IGNORECASE).strip()
    name = name.replace("'", "")
    return name

In [7]:
yahoo_players['cleaned_name'] = yahoo_players['name.full'].apply(clean_name)

In [8]:
master_raw['cleaned_name'] = master_raw['name'].apply(clean_name)

In [9]:
yahoo_players['match_key'] = yahoo_players['cleaned_name'] + '_' + yahoo_players['player_id'].astype('Int64').astype(str)
master_raw['match_key'] = master_raw['cleaned_name'] + '_' + master_raw['stats_id'].astype('Int64').astype(str)

In [10]:
master_raw[master_raw['name'] == 'Jacoby Jones']

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key
7043,8772,ce4d7d70-307f-4093-bf54-432b8b405eb4,9253.0,00-0025460,3689.0,203.0,2495651.0,10517.0,8327.0,4674.0,1220258.0,JoneJa03,NaN,5229.0,4239.0,NaN,8327.0,329337.0,5100.0,NaN,Jacoby Jones,jacoby jones,WR,FA,1984-07-11,36.8,2007.0,3.0,9.0,73.0,TheRealJacoby12,76.0,215.0,Lane,2025,Jacoby Jones,Jacoby Jones_8327


In [11]:
master_raw[master_raw['match_key'].str.contains('_<NA>')]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key
35,17065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Henry Parrish,henry parrish,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,70.0,185.0,Mississippi,2025,Henry Parrish,Henry Parrish_<NA>
37,17067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,John Emery,john emery,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,72.0,220.0,LSU,2025,John Emery,John Emery_<NA>
59,17089,483872b0-2630-11f0-b7f1-df7c830143e7,27218.0,NaN,NaN,12460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18559.0,NaN,NaN,NaN,0.0,26081.0,NaN,Antwane Wells,antwane wells,WR,NYG,2001-04-02,24.5,2025.0,NaN,NaN,NaN,NaN,72.0,205.0,Ole Miss,2025,Antwane Wells,Antwane Wells_<NA>
64,17094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18456.0,NaN,NaN,NaN,0.0,NaN,NaN,Kyren Lacy,kyren lacy,XX,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,74.0,213.0,LSU,2025,Kyren Lacy,Kyren Lacy_<NA>
80,17110,NaN,NaN,NaN,NaN,11594.0,NaN,NaN,NaN,NaN,2965087.0,NaN,NaN,18541.0,NaN,NaN,NaN,0.0,NaN,NaN,Brant Kuithe,brant kuithe,TE,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,74.0,236.0,Utah,2025,Brant Kuithe,Brant Kuithe_<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12139,968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sean Brewer,sean brewer,TE,CIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Sean Brewer,Sean Brewer_<NA>
12140,972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Doug Brien,doug brien,PK,NOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Doug Brien,Doug Brien_<NA>
12141,978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jeremy Brigham,jeremy brigham,TE,OAK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Jeremy Brigham,Jeremy Brigham_<NA>
12142,984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vincent Brisby,vincent brisby,WR,FA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Vincent Brisby,Vincent Brisby_<NA>


In [12]:
print(f"Memory usage: {master_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

Memory usage: 5.25 MB


In [13]:
# Rookies missing are on the yahoo list, will make a yahoo ID map and use to add to master list

yahoo_id_maps = yahoo_players.set_index('match_key')['player_id'].to_dict()


In [14]:
print(yahoo_id_maps['Jacoby Jones_8327'])

8327


In [15]:
master_raw['mapped_ids'] = master_raw['match_key'].map(yahoo_id_maps)
display(master_raw[master_raw['name'] == 'Ben Wooldridge'])
display(master_raw[master_raw['name'] == 'Travis Hunter'])


,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
301,17338,0eba8690-2572-11f0-841c-6faaaac5712d,NaN,NaN,NaN,13065.0,NaN,4373895.0,NaN,NaN,2965115.0,NaN,NaN,18872.0,NaN,NaN,4373895.0,0.0,NaN,NaN,Ben Wooldridge,ben wooldridge,QB,FA,1999-09-27,26.1,2025.0,NaN,NaN,NaN,NaN,74.0,214.0,Louisiana,2025,Ben Wooldridge,Ben Wooldridge_4373895,NaN


,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
44,17074,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025,Travis Hunter,Travis Hunter_41787,NaN


In [16]:
master_raw[(master_raw['mapped_ids'].isna())]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
35,17065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Henry Parrish,henry parrish,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,70.0,185.0,Mississippi,2025,Henry Parrish,Henry Parrish_<NA>,NaN
37,17067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,John Emery,john emery,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,72.0,220.0,LSU,2025,John Emery,John Emery_<NA>,NaN
44,17074,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025,Travis Hunter,Travis Hunter_41787,NaN
59,17089,483872b0-2630-11f0-b7f1-df7c830143e7,27218.0,NaN,NaN,12460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18559.0,NaN,NaN,NaN,0.0,26081.0,NaN,Antwane Wells,antwane wells,WR,NYG,2001-04-02,24.5,2025.0,NaN,NaN,NaN,NaN,72.0,205.0,Ole Miss,2025,Antwane Wells,Antwane Wells_<NA>,NaN
64,17094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18456.0,NaN,NaN,NaN,0.0,NaN,NaN,Kyren Lacy,kyren lacy,XX,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,74.0,213.0,LSU,2025,Kyren Lacy,Kyren Lacy_<NA>,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12139,968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sean Brewer,sean brewer,TE,CIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Sean Brewer,Sean Brewer_<NA>,NaN
12140,972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Doug Brien,doug brien,PK,NOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Doug Brien,Doug Brien_<NA>,NaN
12141,978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jeremy Brigham,jeremy brigham,TE,OAK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Jeremy Brigham,Jeremy Brigham_<NA>,NaN
12142,984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vincent Brisby,vincent brisby,WR,FA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Vincent Brisby,Vincent Brisby_<NA>,NaN


In [17]:
# Count NaN in mapped_ids
print("NaN in mapped_ids:", master_raw['mapped_ids'].isna().sum())

# Check if mismatch should include NaN
print("Should show mismatches:", (~master_raw['mapped_ids'].isin(master_raw['stats_id'])).sum())

# Are the NaN values in mapped_ids also NaN in stats_id?
both_nan = master_raw['mapped_ids'].isna() & master_raw['stats_id'].isna()
print("Rows where BOTH are NaN:", both_nan.sum())

NaN in mapped_ids: 2601
Should show mismatches: 0
Rows where BOTH are NaN: 1093


In [18]:
master_raw[master_raw['name'] == 'Adam Vinatieri']

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
9688,2842,9ecf8040-10f9-4a5c-92da-1b4d77bd6760,9004.0,00-0016919,226.0,120.0,2503471.0,1097.0,3727.0,2074.0,1392.0,vinatada01,NaN,395.0,1152.0,NaN,3727.0,23849.0,3258.0,NaN,Adam Vinatieri,adam vinatieri,PK,FA,1972-12-28,49.2,1996.0,NaN,NaN,NaN,aVinatieri4,72.0,212.0,South Dakota State,2025,Adam Vinatieri,Adam Vinatieri_3727,3727.0


In [19]:
# Exclude rows where BOTH are NaN (those are fine)
# Show rows where they don't match
master_mismatch = master_raw[
    (master_raw['mapped_ids'] != master_raw['stats_id']) & 
    ~(master_raw['mapped_ids'].isna() & master_raw['stats_id'].isna())
]
print(f"True mismatches: {len(master_mismatch)}")

True mismatches: 1508


In [20]:
master_mismatch.head()

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
44,17074,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025,Travis Hunter,Travis Hunter_41787,NaN
190,17220,80dce1ba-c997-47e6-a37e-e92fefcbdc1b,NaN,00-0040200,143934.0,12713.0,58387.0,4569923.0,NaN,NaN,26699665.0,BorrAn00,NaN,18743.0,NaN,NaN,41980.0,0.0,26106.0,NaN,Andres Borregales,andres borregales,PK,NEP,2002-01-02,23.8,2025.0,6.0,6.0,182.0,NaN,71.0,202.0,Miami (FL),2025,Andres Borregales,Andres Borregales_41980,NaN
294,17329,826620a0-249a-11f0-9302-cb167d81eff7,23170.0,00-0040332,123119.0,12479.0,58768.0,4690972.0,NaN,NaN,3168195.0,JackJa05,NaN,18527.0,NaN,NaN,4690972.0,0.0,25901.0,NaN,Ja'Quinden Jackson,jaquinden jackson,RB,JAC,2001-09-12,24.1,2025.0,NaN,NaN,NaN,NaN,74.0,233.0,Arkansas,2025,JaQuinden Jackson,JaQuinden Jackson_4690972,NaN
301,17338,0eba8690-2572-11f0-841c-6faaaac5712d,NaN,NaN,NaN,13065.0,NaN,4373895.0,NaN,NaN,2965115.0,NaN,NaN,18872.0,NaN,NaN,4373895.0,0.0,NaN,NaN,Ben Wooldridge,ben wooldridge,QB,FA,1999-09-27,26.1,2025.0,NaN,NaN,NaN,NaN,74.0,214.0,Louisiana,2025,Ben Wooldridge,Ben Wooldridge_4373895,NaN
358,16590,ec7b8cf0-0bb9-11ef-bb45-99966e93bb71,23002.0,NaN,NaN,12259.0,NaN,NaN,NaN,NaN,3123935.0,NaN,NaN,16999.0,NaN,NaN,41559.0,0.0,NaN,NaN,Taulia Tagovailoa,taulia tagovailoa,QB,FA,2000-02-15,25.7,2024.0,NaN,NaN,NaN,NaN,71.0,200.0,Maryland,2025,Taulia Tagovailoa,Taulia Tagovailoa_41559,NaN


In [21]:
master_mismatch[master_mismatch['stats_id'] > 101000]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
294,17329,826620a0-249a-11f0-9302-cb167d81eff7,23170.0,00-0040332,123119.0,12479.0,58768.0,4690972.0,NaN,NaN,3168195.0,JackJa05,NaN,18527.0,NaN,NaN,4690972.0,0.0,25901.0,NaN,Ja'Quinden Jackson,jaquinden jackson,RB,JAC,2001-09-12,24.1,2025.0,NaN,NaN,NaN,NaN,74.0,233.0,Arkansas,2025,JaQuinden Jackson,JaQuinden Jackson_4690972,NaN
301,17338,0eba8690-2572-11f0-841c-6faaaac5712d,NaN,NaN,NaN,13065.0,NaN,4373895.0,NaN,NaN,2965115.0,NaN,NaN,18872.0,NaN,NaN,4373895.0,0.0,NaN,NaN,Ben Wooldridge,ben wooldridge,QB,FA,1999-09-27,26.1,2025.0,NaN,NaN,NaN,NaN,74.0,214.0,Louisiana,2025,Ben Wooldridge,Ben Wooldridge_4373895,NaN
666,16964,9f2c6ea0-0570-11ef-af69-dfa5fe884ba6,NaN,00-0039614,141559.0,11897.0,57742.0,4698301.0,NaN,NaN,26717777.0,JackJe01,NaN,18104.0,NaN,NaN,4698301.0,0.0,NaN,1274459.0,Jermaine Jackson,jermaine jackson,WR,FA,2000-06-02,25.4,2024.0,NaN,NaN,NaN,NaN,67.0,170.0,Idaho,2025,Jermaine Jackson,Jermaine Jackson_4698301,NaN


These three player shave no stats, will drop

In [22]:
master_mismatch = master_mismatch[master_mismatch['stats_id'] < 101000]
master_mismatch[master_mismatch['stats_id'] > 101000]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids


In [23]:
master_mismatch.loc[(master_mismatch['stats_id'].isna()), 'stats_id'] = master_mismatch.loc[(master_mismatch['stats_id'].isna()), 'mapped_ids']

In [24]:
master_mismatch[master_mismatch['stats_id'].isna()]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids


In [25]:
master_mismatch[master_mismatch['stats_id'].isna()]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids


In [26]:
master_mismatch.head()

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
44,17074,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025,Travis Hunter,Travis Hunter_41787,NaN
190,17220,80dce1ba-c997-47e6-a37e-e92fefcbdc1b,NaN,00-0040200,143934.0,12713.0,58387.0,4569923.0,NaN,NaN,26699665.0,BorrAn00,NaN,18743.0,NaN,NaN,41980.0,0.0,26106.0,NaN,Andres Borregales,andres borregales,PK,NEP,2002-01-02,23.8,2025.0,6.0,6.0,182.0,NaN,71.0,202.0,Miami (FL),2025,Andres Borregales,Andres Borregales_41980,NaN
358,16590,ec7b8cf0-0bb9-11ef-bb45-99966e93bb71,23002.0,NaN,NaN,12259.0,NaN,NaN,NaN,NaN,3123935.0,NaN,NaN,16999.0,NaN,NaN,41559.0,0.0,NaN,NaN,Taulia Tagovailoa,taulia tagovailoa,QB,FA,2000-02-15,25.7,2024.0,NaN,NaN,NaN,NaN,71.0,200.0,Maryland,2025,Taulia Tagovailoa,Taulia Tagovailoa_41559,NaN
585,16818,8e6fc170-07d2-11ef-95d0-7db2ea91136a,NaN,NaN,NaN,12188.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41477.0,0.0,NaN,NaN,Justin Strong,justin strong,RB,FA,NaN,NaN,2024.0,NaN,NaN,NaN,NaN,72.0,205.0,Southern Illinois,2025,Justin Strong,Justin Strong_41477,NaN
600,16833,fffee080-04ff-11ef-8a9a-5f3b29e3b4c8,26405.0,00-0039541,124169.0,11886.0,57660.0,4429153.0,41163.0,NaN,3171022.0,GrifLi00,NaN,17973.0,NaN,NaN,41163.0,0.0,NaN,1231735.0,Lideatrick Griffin,lideatrick griffin,WR,FA,2001-01-23,24.7,2024.0,NaN,NaN,NaN,NaN,70.0,181.0,Mississippi State,2025,Lideatrick Griffin,Lideatrick Griffin_41163,NaN


In [27]:
master_mismatch.loc[master_mismatch['stats_id'].duplicated(keep=False)]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids


In [28]:
master_mismatch.loc[master_mismatch['stats_id'] < 3000].head()

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
9848,1032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BantBr00,bradford-banta-1,837.0,NaN,NaN,2732.0,0.0,NaN,NaN,Bradford Banta,bradford banta,TE,FA,1970-12-14,50.4,1994.0,4.0,3.0,106.0,NaN,78.0,253.0,Southern California,2025,Bradford Banta,Bradford Banta_2732,NaN
9852,1076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,briendou01,doug-brien-1,849.0,NaN,NaN,2935.0,0.0,NaN,NaN,Doug Brien,doug brien,PK,FA,1970-11-24,50.5,1994.0,3.0,20.0,85.0,NaN,NaN,185.0,California,2025,Doug Brien,Doug Brien_2935,NaN
9854,1095,NaN,NaN,00-0002099,NaN,NaN,NaN,658.0,NaN,322.0,1294.0,BrucIs00,isaac-bruce-1,626.0,414.0,NaN,2914.0,0.0,NaN,NaN,Isaac Bruce,isaac bruce,WR,FA,1972-11-10,48.5,1994.0,2.0,4.0,33.0,NaN,72.0,188.0,Memphis,2025,Isaac Bruce,Isaac Bruce_2914,NaN
9858,1183,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1065.0,DilfTr00,trent-dilfer-1,666.0,552.0,NaN,2900.0,0.0,NaN,NaN,Trent Dilfer,trent dilfer,QB,FA,1972-03-13,49.2,1994.0,1.0,6.0,6.0,DilfersDimes,76.0,234.0,Fresno State,2025,Trent Dilfer,Trent Dilfer_2900,NaN
9861,1213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FaulMa00,marshall-faulk-1,292.0,368.0,NaN,2728.0,0.0,NaN,NaN,Marshall Faulk,marshall faulk,RB,FA,1973-02-26,48.2,1994.0,1.0,2.0,2.0,marshallfaulk,70.0,211.0,NaN,2025,Marshall Faulk,Marshall Faulk_2728,NaN


In [29]:
master_mismatch = master_mismatch.drop(labels=master_mismatch.loc[master_mismatch['stats_id'] < 3726].index, axis=0)
master_mismatch['stats_id'].min()

np.float64(3731.0)

In [30]:
master_mismatch[~master_mismatch['mapped_ids'].isin(master_mismatch['stats_id'])]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
44,17074,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025,Travis Hunter,Travis Hunter_41787,NaN
190,17220,80dce1ba-c997-47e6-a37e-e92fefcbdc1b,NaN,00-0040200,143934.0,12713.0,58387.0,4569923.0,NaN,NaN,26699665.0,BorrAn00,NaN,18743.0,NaN,NaN,41980.0,0.0,26106.0,NaN,Andres Borregales,andres borregales,PK,NEP,2002-01-02,23.8,2025.0,6.0,6.0,182.0,NaN,71.0,202.0,Miami (FL),2025,Andres Borregales,Andres Borregales_41980,NaN
358,16590,ec7b8cf0-0bb9-11ef-bb45-99966e93bb71,23002.0,NaN,NaN,12259.0,NaN,NaN,NaN,NaN,3123935.0,NaN,NaN,16999.0,NaN,NaN,41559.0,0.0,NaN,NaN,Taulia Tagovailoa,taulia tagovailoa,QB,FA,2000-02-15,25.7,2024.0,NaN,NaN,NaN,NaN,71.0,200.0,Maryland,2025,Taulia Tagovailoa,Taulia Tagovailoa_41559,NaN
585,16818,8e6fc170-07d2-11ef-95d0-7db2ea91136a,NaN,NaN,NaN,12188.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41477.0,0.0,NaN,NaN,Justin Strong,justin strong,RB,FA,NaN,NaN,2024.0,NaN,NaN,NaN,NaN,72.0,205.0,Southern Illinois,2025,Justin Strong,Justin Strong_41477,NaN
600,16833,fffee080-04ff-11ef-8a9a-5f3b29e3b4c8,26405.0,00-0039541,124169.0,11886.0,57660.0,4429153.0,41163.0,NaN,3171022.0,GrifLi00,NaN,17973.0,NaN,NaN,41163.0,0.0,NaN,1231735.0,Lideatrick Griffin,lideatrick griffin,WR,FA,2001-01-23,24.7,2024.0,NaN,NaN,NaN,NaN,70.0,181.0,Mississippi State,2025,Lideatrick Griffin,Lideatrick Griffin_41163,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12034,9793,20af511b-03b7-4905-bc41-158f286b99c6,NaN,NaN,NaN,2864.0,2553937.0,12899.0,29138.0,NaN,2175732.0,NaN,NaN,6002.0,5243.0,NaN,9736.0,245814.0,17332.0,NaN,Mike Reilly,mike reilly,QB,STL,1985-01-25,36.3,0.0,NaN,NaN,NaN,NaN,76.0,240.0,Central Washington,2025,Mike Reilly,Mike Reilly_9736,NaN
12036,9796,NaN,NaN,00-0026792,NaN,NaN,NaN,13027.0,NaN,NaN,1675238.0,WillPa01,NaN,6401.0,5595.0,NaN,9721.0,267416.0,NaN,NaN,Patrick Williams,patrick williams,WR,BAL,1986-01-13,35.3,0.0,NaN,NaN,NaN,NaN,73.0,204.0,Colorado,2025,Patrick Williams,Patrick Williams_9721,NaN
12038,9800,NaN,NaN,00-0024825,NaN,NaN,NaN,11142.0,NaN,NaN,1226291.0,NordJa00,NaN,5467.0,4608.0,NaN,8692.0,219960.0,NaN,NaN,Jake Nordin,jake nordin,TE,DET,1984-07-08,36.8,0.0,NaN,NaN,NaN,NaN,75.0,255.0,Northern Illinois,2025,Jake Nordin,Jake Nordin_8692,NaN
12039,9806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6202.0,5498.0,NaN,9122.0,463275.0,NaN,NaN,Sam Giguere,sam giguere,WR,FA,1985-07-11,35.8,0.0,NaN,NaN,NaN,NaN,71.0,215.0,Sherbrooke,2025,Sam Giguere,Sam Giguere_9122,NaN


In [31]:
print(master_mismatch['mapped_ids'].isna().sum())

1409


In [32]:
master_mismatch.shape

(1409, 38)

In [33]:
master_mismatch.loc[(master_mismatch['mapped_ids'].isna()), 'mapped_ids'] = master_mismatch.loc[(master_mismatch['mapped_ids'].isna()), 'stats_id']
master_mismatch[master_mismatch['name'] == 'Ollie Gordon']

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids


In [34]:
master_mismatch.head()

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
44,17074,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025,Travis Hunter,Travis Hunter_41787,41787.0
190,17220,80dce1ba-c997-47e6-a37e-e92fefcbdc1b,NaN,00-0040200,143934.0,12713.0,58387.0,4569923.0,NaN,NaN,26699665.0,BorrAn00,NaN,18743.0,NaN,NaN,41980.0,0.0,26106.0,NaN,Andres Borregales,andres borregales,PK,NEP,2002-01-02,23.8,2025.0,6.0,6.0,182.0,NaN,71.0,202.0,Miami (FL),2025,Andres Borregales,Andres Borregales_41980,41980.0
358,16590,ec7b8cf0-0bb9-11ef-bb45-99966e93bb71,23002.0,NaN,NaN,12259.0,NaN,NaN,NaN,NaN,3123935.0,NaN,NaN,16999.0,NaN,NaN,41559.0,0.0,NaN,NaN,Taulia Tagovailoa,taulia tagovailoa,QB,FA,2000-02-15,25.7,2024.0,NaN,NaN,NaN,NaN,71.0,200.0,Maryland,2025,Taulia Tagovailoa,Taulia Tagovailoa_41559,41559.0
585,16818,8e6fc170-07d2-11ef-95d0-7db2ea91136a,NaN,NaN,NaN,12188.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41477.0,0.0,NaN,NaN,Justin Strong,justin strong,RB,FA,NaN,NaN,2024.0,NaN,NaN,NaN,NaN,72.0,205.0,Southern Illinois,2025,Justin Strong,Justin Strong_41477,41477.0
600,16833,fffee080-04ff-11ef-8a9a-5f3b29e3b4c8,26405.0,00-0039541,124169.0,11886.0,57660.0,4429153.0,41163.0,NaN,3171022.0,GrifLi00,NaN,17973.0,NaN,NaN,41163.0,0.0,NaN,1231735.0,Lideatrick Griffin,lideatrick griffin,WR,FA,2001-01-23,24.7,2024.0,NaN,NaN,NaN,NaN,70.0,181.0,Mississippi State,2025,Lideatrick Griffin,Lideatrick Griffin_41163,41163.0


In [35]:
updated_yahoo_maps = master_mismatch.set_index('match_key')['stats_id'].to_dict()
display(updated_yahoo_maps)

{'Travis Hunter_41787': 41787.0,
 'Andres Borregales_41980': 41980.0,
 'Taulia Tagovailoa_41559': 41559.0,
 'Justin Strong_41477': 41477.0,
 'Lideatrick Griffin_41163': 41163.0,
 'CJ Johnson_40461': 40461.0,
 'Scott Matlock_40225': 40225.0,
 'Demario Douglas_40222': 40222.0,
 'Grant Dubose_40251': 40251.0,
 'Keytaon Thompson_40592': 40592.0,
 'Mitchell Tinsley_40313': 40313.0,
 'EJ Perry_34346': 34346.0,
 'Dvonte Price_34521': 34521.0,
 'TJ Pledger_34691': 34691.0,
 'John Metchie_34063': 34063.0,
 'Chigoziem Okonkwo_34099': 34099.0,
 'Andrew Ogletree_34148': 34148.0,
 'Lujuan Winningham_34255': 34255.0,
 'John Parker Romo_34657': 34657.0,
 'Mike Strachan_33620': 33620.0,
 'BJ Emmons_33855': 33855.0,
 'AJ Dillon_32732': 32732.0,
 'Lamical Perine_32790': 32790.0,
 'Gabriel Davis_32798': 32798.0,
 'K.J. Hill_32890': 32890.0,
 'Jacob Breeland_32994': 32994.0,
 'LJ Scott_32569': 32569.0,
 'Marquise Brown_31857': 31857.0,
 'JJ Arcega-Whiteside_31889': 31889.0,
 'Tyron Johnson_32511': 32511.0

In [36]:
master_raw[master_raw['mapped_ids'].isna()].head()

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
35,17065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Henry Parrish,henry parrish,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,70.0,185.0,Mississippi,2025,Henry Parrish,Henry Parrish_<NA>,NaN
37,17067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,John Emery,john emery,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,72.0,220.0,LSU,2025,John Emery,John Emery_<NA>,NaN
44,17074,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025,Travis Hunter,Travis Hunter_41787,NaN
59,17089,483872b0-2630-11f0-b7f1-df7c830143e7,27218.0,NaN,NaN,12460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18559.0,NaN,NaN,NaN,0.0,26081.0,NaN,Antwane Wells,antwane wells,WR,NYG,2001-04-02,24.5,2025.0,NaN,NaN,NaN,NaN,72.0,205.0,Ole Miss,2025,Antwane Wells,Antwane Wells_<NA>,NaN
64,17094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18456.0,NaN,NaN,NaN,0.0,NaN,NaN,Kyren Lacy,kyren lacy,XX,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,74.0,213.0,LSU,2025,Kyren Lacy,Kyren Lacy_<NA>,NaN


In [37]:
master_raw = master_raw[((master_raw['stats_id'] < 101000) & (master_raw['stats_id'] > 3720)) | (master_raw['stats_id'].isna())]

In [38]:
master_raw[master_raw['mapped_ids'].isna()].head()

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
35,17065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Henry Parrish,henry parrish,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,70.0,185.0,Mississippi,2025,Henry Parrish,Henry Parrish_<NA>,NaN
37,17067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,John Emery,john emery,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,72.0,220.0,LSU,2025,John Emery,John Emery_<NA>,NaN
44,17074,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025,Travis Hunter,Travis Hunter_41787,NaN
59,17089,483872b0-2630-11f0-b7f1-df7c830143e7,27218.0,NaN,NaN,12460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18559.0,NaN,NaN,NaN,0.0,26081.0,NaN,Antwane Wells,antwane wells,WR,NYG,2001-04-02,24.5,2025.0,NaN,NaN,NaN,NaN,72.0,205.0,Ole Miss,2025,Antwane Wells,Antwane Wells_<NA>,NaN
64,17094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18456.0,NaN,NaN,NaN,0.0,NaN,NaN,Kyren Lacy,kyren lacy,XX,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,74.0,213.0,LSU,2025,Kyren Lacy,Kyren Lacy_<NA>,NaN


In [39]:
master_raw[master_raw['name'] == 'Antwane Wells']

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
59,17089,483872b0-2630-11f0-b7f1-df7c830143e7,27218.0,NaN,NaN,12460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18559.0,NaN,NaN,NaN,0.0,26081.0,NaN,Antwane Wells,antwane wells,WR,NYG,2001-04-02,24.5,2025.0,NaN,NaN,NaN,NaN,72.0,205.0,Ole Miss,2025,Antwane Wells,Antwane Wells_<NA>,NaN


In [40]:
master_raw.loc[(master_raw['match_key'].isin(updated_yahoo_maps.keys())) & (master_raw['mapped_ids'].isna()), 'mapped_ids'] = master_raw.loc[(master_raw['match_key'].isin(updated_yahoo_maps.keys())) & (master_raw['mapped_ids'].isna()), 'match_key'].map(updated_yahoo_maps)

In [41]:
master_raw[master_raw['mapped_ids'].isna()].head()

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
35,17065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Henry Parrish,henry parrish,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,70.0,185.0,Mississippi,2025,Henry Parrish,Henry Parrish_<NA>,NaN
37,17067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,John Emery,john emery,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,72.0,220.0,LSU,2025,John Emery,John Emery_<NA>,NaN
59,17089,483872b0-2630-11f0-b7f1-df7c830143e7,27218.0,NaN,NaN,12460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18559.0,NaN,NaN,NaN,0.0,26081.0,NaN,Antwane Wells,antwane wells,WR,NYG,2001-04-02,24.5,2025.0,NaN,NaN,NaN,NaN,72.0,205.0,Ole Miss,2025,Antwane Wells,Antwane Wells_<NA>,NaN
64,17094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18456.0,NaN,NaN,NaN,0.0,NaN,NaN,Kyren Lacy,kyren lacy,XX,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,74.0,213.0,LSU,2025,Kyren Lacy,Kyren Lacy_<NA>,NaN
80,17110,NaN,NaN,NaN,NaN,11594.0,NaN,NaN,NaN,NaN,2965087.0,NaN,NaN,18541.0,NaN,NaN,NaN,0.0,NaN,NaN,Brant Kuithe,brant kuithe,TE,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,74.0,236.0,Utah,2025,Brant Kuithe,Brant Kuithe_<NA>,NaN


In [42]:
master_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5432 entries, 0 to 12143
Data columns (total 38 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   mfl_id            5432 non-null   int64  
 1   sportradar_id     3263 non-null   object 
 2   fantasypros_id    2489 non-null   float64
 3   gsis_id           3339 non-null   object 
 4   pff_id            3146 non-null   float64
 5   sleeper_id        2844 non-null   float64
 6   nfl_id            3091 non-null   float64
 7   espn_id           3457 non-null   float64
 8   yahoo_id          2684 non-null   float64
 9   fleaflicker_id    493 non-null    float64
 10  cbs_id            4130 non-null   float64
 11  pfr_id            3917 non-null   object 
 12  cfbref_id         2171 non-null   object 
 13  rotowire_id       4749 non-null   float64
 14  rotoworld_id      3257 non-null   float64
 15  ktc_id            394 non-null    float64
 16  stats_id          4339 non-null   float64
 17 

In [43]:
master_raw[(master_raw['cleaned_name'].isin(yahoo_players['cleaned_name'])) & (master_raw['stats_id'].isna())]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
59,17089,483872b0-2630-11f0-b7f1-df7c830143e7,27218.0,NaN,NaN,12460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18559.0,NaN,NaN,NaN,0.0,26081.0,NaN,Antwane Wells,antwane wells,WR,NYG,2001-04-02,24.5,2025.0,NaN,NaN,NaN,NaN,72.0,205.0,Ole Miss,2025,Antwane Wells,Antwane Wells_<NA>,NaN
141,17171,1e95c7fc-ecf7-4fcb-82d8-d77e2b6872d5,27315.0,00-0040705,129918.0,12540.0,58308.0,4431268.0,NaN,NaN,3162452.0,DikeCh00,NaN,18641.0,NaN,1814.0,NaN,0.0,25987.0,NaN,Chimere Dike,chimere dike,WR,TEN,2001-12-14,23.8,2025.0,4.0,1.0,103.0,NaN,73.0,196.0,Florida,2025,Chimere Dike,Chimere Dike_<NA>,NaN
245,17277,7bf5a2e0-2512-11f0-9825-33176da47896,27261.0,00-0040279,99867.0,12961.0,58715.0,4568263.0,NaN,NaN,NaN,FitzRy01,NaN,18508.0,NaN,NaN,NaN,0.0,26108.0,NaN,Ryan Fitzgerald,ryan fitzgerald,PK,CAR,2000-08-13,25.2,2025.0,NaN,NaN,NaN,NaN,71.0,193.0,Florida State,2025,Ryan Fitzgerald,Ryan Fitzgerald_<NA>,NaN
344,17396,NaN,NaN,00-0040534,123406.0,NaN,58980.0,4570738.0,NaN,NaN,NaN,WillKe08,NaN,18841.0,NaN,NaN,NaN,0.0,NaN,NaN,Ke'Shawn Williams,keshawn williams,WR,PIT,2001-12-27,23.8,2025.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,KeShawn Williams,KeShawn Williams_<NA>,NaN
402,16634,ffd59710-0582-11ef-823c-c3942ac17525,23957.0,00-0039521,101130.0,11921.0,57639.0,4428811.0,41202.0,NaN,3131562.0,WeavXa00,NaN,18041.0,NaN,NaN,NaN,0.0,25032.0,1180538.0,Xavier Weaver,xavier weaver,WR,ARI,2000-10-12,25.0,2024.0,NaN,NaN,NaN,NaN,73.0,185.0,Colorado,2025,Xavier Weaver,Xavier Weaver_<NA>,NaN
406,16638,NaN,NaN,NaN,NaN,NaN,NaN,4427225.0,NaN,NaN,NaN,NaN,NaN,17006.0,NaN,NaN,NaN,0.0,NaN,NaN,Zakhari Franklin,zakhari franklin,WR,FA,NaN,NaN,2024.0,NaN,NaN,NaN,NaN,NaN,NaN,Mississippi,2025,Zakhari Franklin,Zakhari Franklin_<NA>,NaN
741,16195,ea3d1393-8c3b-4e81-a519-f636d28d692e,24354.0,00-0038393,99711.0,9486.0,56024.0,4428850.0,40201.0,NaN,3122375.0,WickDo00,dontayvion-wicks-1,16658.0,NaN,1448.0,NaN,0.0,23253.0,1172428.0,Dontayvion Wicks,dontayvion wicks,WR,GBP,2001-06-16,24.3,2023.0,5.0,24.0,159.0,NaN,73.0,206.0,Virginia,2025,Dontayvion Wicks,Dontayvion Wicks_<NA>,NaN
766,16220,5d4a5e4f-7c74-4b4c-b8e5-33bb831e3057,23821.0,00-0038394,77888.0,10220.0,56027.0,4362523.0,40180.0,NaN,2962073.0,MallWi00,will-mallory-1,17014.0,NaN,NaN,NaN,0.0,23143.0,1121873.0,Will Mallory,will mallory,TE,IND,1999-06-22,26.3,2023.0,5.0,27.0,162.0,NaN,76.0,239.0,Miami (FL),2025,Will Mallory,Will Mallory_<NA>,NaN
2271,14091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Tre Watson,tre watson,RB,FA,NaN,NaN,2019.0,NaN,NaN,NaN,NaN,71.0,195.0,Texas,2025,Tre Watson,Tre Watson_<NA>,NaN
2679,16859,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2180626.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Tyron Billy-Johnson,tyron billy-johnson,XX,FA,1996-01-08,29.1,2019.0,NaN,NaN,NaN,NaN,73.0,193.0,Oklahoma State,2025,Tyron Billy-Johnson,Tyron Billy-Johnson_<NA>,NaN


START HERE -- YOU WERE FIGURING OUT IF YOU COULD DROP TEH DUPLIACTES AND IT LOOKS LIKE YOU CAN

In [44]:
master_raw = master_raw[~((master_raw['cleaned_name'].duplicated(keep=False)) & (master_raw['mapped_ids'].isna()))]
master_raw[master_raw['mapped_ids'].isna()].head()

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
35,17065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Henry Parrish,henry parrish,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,70.0,185.0,Mississippi,2025,Henry Parrish,Henry Parrish_<NA>,NaN
37,17067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,John Emery,john emery,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,72.0,220.0,LSU,2025,John Emery,John Emery_<NA>,NaN
59,17089,483872b0-2630-11f0-b7f1-df7c830143e7,27218.0,NaN,NaN,12460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18559.0,NaN,NaN,NaN,0.0,26081.0,NaN,Antwane Wells,antwane wells,WR,NYG,2001-04-02,24.5,2025.0,NaN,NaN,NaN,NaN,72.0,205.0,Ole Miss,2025,Antwane Wells,Antwane Wells_<NA>,NaN
64,17094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18456.0,NaN,NaN,NaN,0.0,NaN,NaN,Kyren Lacy,kyren lacy,XX,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,74.0,213.0,LSU,2025,Kyren Lacy,Kyren Lacy_<NA>,NaN
80,17110,NaN,NaN,NaN,NaN,11594.0,NaN,NaN,NaN,NaN,2965087.0,NaN,NaN,18541.0,NaN,NaN,NaN,0.0,NaN,NaN,Brant Kuithe,brant kuithe,TE,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,74.0,236.0,Utah,2025,Brant Kuithe,Brant Kuithe_<NA>,NaN


In [45]:
master_raw.loc[(master_raw['mapped_ids'].isna()) & (master_raw['yahoo_id'].notna()), 'mapped_ids'] = master_raw.loc[(master_raw['mapped_ids'].isna()) & (master_raw['yahoo_id'].notna()), 'yahoo_id']
master_raw[master_raw['mapped_ids'].isna()]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
35,17065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Henry Parrish,henry parrish,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,70.0,185.0,Mississippi,2025,Henry Parrish,Henry Parrish_<NA>,NaN
37,17067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,John Emery,john emery,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,72.0,220.0,LSU,2025,John Emery,John Emery_<NA>,NaN
59,17089,483872b0-2630-11f0-b7f1-df7c830143e7,27218.0,NaN,NaN,12460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18559.0,NaN,NaN,NaN,0.0,26081.0,NaN,Antwane Wells,antwane wells,WR,NYG,2001-04-02,24.5,2025.0,NaN,NaN,NaN,NaN,72.0,205.0,Ole Miss,2025,Antwane Wells,Antwane Wells_<NA>,NaN
64,17094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18456.0,NaN,NaN,NaN,0.0,NaN,NaN,Kyren Lacy,kyren lacy,XX,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,74.0,213.0,LSU,2025,Kyren Lacy,Kyren Lacy_<NA>,NaN
80,17110,NaN,NaN,NaN,NaN,11594.0,NaN,NaN,NaN,NaN,2965087.0,NaN,NaN,18541.0,NaN,NaN,NaN,0.0,NaN,NaN,Brant Kuithe,brant kuithe,TE,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,74.0,236.0,Utah,2025,Brant Kuithe,Brant Kuithe_<NA>,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12107,792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jeff Blake,jeff blake,QB,NOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Jeff Blake,Jeff Blake_<NA>,NaN
12109,804,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Drew Bledsoe,drew bledsoe,QB,NEP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Drew Bledsoe,Drew Bledsoe_<NA>,NaN
12127,902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R.j. Bowers,rj bowers,RB,CAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,R.j. Bowers,R.j. Bowers_<NA>,NaN
12134,942,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kyle Brady,kyle brady,TE,JAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Kyle Brady,Kyle Brady_<NA>,NaN


In [46]:
master_raw['mapped_ids'].isna().sum()

np.int64(602)

In [47]:
master_raw.loc[(master_raw['mapped_ids'].isna()) & (master_raw['cleaned_name'].isin(yahoo_players['cleaned_name']))]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
59,17089,483872b0-2630-11f0-b7f1-df7c830143e7,27218.0,NaN,NaN,12460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18559.0,NaN,NaN,NaN,0.0,26081.0,NaN,Antwane Wells,antwane wells,WR,NYG,2001-04-02,24.5,2025.0,NaN,NaN,NaN,NaN,72.0,205.0,Ole Miss,2025,Antwane Wells,Antwane Wells_<NA>,NaN
141,17171,1e95c7fc-ecf7-4fcb-82d8-d77e2b6872d5,27315.0,00-0040705,129918.0,12540.0,58308.0,4431268.0,NaN,NaN,3162452.0,DikeCh00,NaN,18641.0,NaN,1814.0,NaN,0.0,25987.0,NaN,Chimere Dike,chimere dike,WR,TEN,2001-12-14,23.8,2025.0,4.0,1.0,103.0,NaN,73.0,196.0,Florida,2025,Chimere Dike,Chimere Dike_<NA>,NaN
245,17277,7bf5a2e0-2512-11f0-9825-33176da47896,27261.0,00-0040279,99867.0,12961.0,58715.0,4568263.0,NaN,NaN,NaN,FitzRy01,NaN,18508.0,NaN,NaN,NaN,0.0,26108.0,NaN,Ryan Fitzgerald,ryan fitzgerald,PK,CAR,2000-08-13,25.2,2025.0,NaN,NaN,NaN,NaN,71.0,193.0,Florida State,2025,Ryan Fitzgerald,Ryan Fitzgerald_<NA>,NaN
344,17396,NaN,NaN,00-0040534,123406.0,NaN,58980.0,4570738.0,NaN,NaN,NaN,WillKe08,NaN,18841.0,NaN,NaN,NaN,0.0,NaN,NaN,Ke'Shawn Williams,keshawn williams,WR,PIT,2001-12-27,23.8,2025.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,KeShawn Williams,KeShawn Williams_<NA>,NaN
406,16638,NaN,NaN,NaN,NaN,NaN,NaN,4427225.0,NaN,NaN,NaN,NaN,NaN,17006.0,NaN,NaN,NaN,0.0,NaN,NaN,Zakhari Franklin,zakhari franklin,WR,FA,NaN,NaN,2024.0,NaN,NaN,NaN,NaN,NaN,NaN,Mississippi,2025,Zakhari Franklin,Zakhari Franklin_<NA>,NaN
2271,14091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Tre Watson,tre watson,RB,FA,NaN,NaN,2019.0,NaN,NaN,NaN,NaN,71.0,195.0,Texas,2025,Tre Watson,Tre Watson_<NA>,NaN
2679,16859,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2180626.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Tyron Billy-Johnson,tyron billy-johnson,XX,FA,1996-01-08,29.1,2019.0,NaN,NaN,NaN,NaN,73.0,193.0,Oklahoma State,2025,Tyron Billy-Johnson,Tyron Billy-Johnson_<NA>,NaN
11166,4819,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2371.0,NaN,NaN,NaN,0.0,NaN,NaN,Chris Coleman,chris coleman,WR,FA,1977-05-08,44.0,0.0,NaN,NaN,NaN,NaN,72.0,202.0,North Carolina State,2025,Chris Coleman,Chris Coleman_<NA>,NaN


In [48]:
master_raw.loc[(master_raw['mapped_ids'].isna()) & (master_raw['cleaned_name'].isin(yahoo_players['cleaned_name'])),
               'mapped_ids'] = master_raw.loc[(master_raw['mapped_ids'].isna()) & (master_raw['cleaned_name'].isin(yahoo_players['cleaned_name'])), 'cleaned_name'].map((yahoo_players[~yahoo_players['cleaned_name'].duplicated(keep=False)]).set_index('cleaned_name')['player_id'])

master_raw[master_raw['cleaned_name'] == 'Ryan Fitzgerald']

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
245,17277,7bf5a2e0-2512-11f0-9825-33176da47896,27261.0,00-0040279,99867.0,12961.0,58715.0,4568263.0,NaN,NaN,NaN,FitzRy01,NaN,18508.0,NaN,NaN,NaN,0.0,26108.0,NaN,Ryan Fitzgerald,ryan fitzgerald,PK,CAR,2000-08-13,25.2,2025.0,NaN,NaN,NaN,NaN,71.0,193.0,Florida State,2025,Ryan Fitzgerald,Ryan Fitzgerald_<NA>,42262.0


In [49]:
master_raw.loc[master_raw['mapped_ids'].isna() & (master_raw['cleaned_name'].isin(yahoo_players['cleaned_name'])), 'mapped_ids'] = 32511

In [50]:
master_raw[master_raw['mapped_ids'].isna() & (master_raw['cleaned_name'].isin(yahoo_players['cleaned_name']))]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids


In [51]:
master_raw = master_raw[master_raw['mapped_ids'].notna()]
master_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4356 entries, 0 to 12040
Data columns (total 38 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   mfl_id            4356 non-null   int64  
 1   sportradar_id     3260 non-null   object 
 2   fantasypros_id    2489 non-null   float64
 3   gsis_id           3339 non-null   object 
 4   pff_id            3146 non-null   float64
 5   sleeper_id        2837 non-null   float64
 6   nfl_id            3091 non-null   float64
 7   espn_id           3456 non-null   float64
 8   yahoo_id          2684 non-null   float64
 9   fleaflicker_id    493 non-null    float64
 10  cbs_id            4096 non-null   float64
 11  pfr_id            3537 non-null   object 
 12  cfbref_id         1843 non-null   object 
 13  rotowire_id       4243 non-null   float64
 14  rotoworld_id      3203 non-null   float64
 15  ktc_id            394 non-null    float64
 16  stats_id          4339 non-null   float64
 17 

In [52]:
master_raw.loc[master_raw['match_key'].str.contains('<NA>'), 'match_key'] = master_raw.loc[master_raw['match_key'].str.contains('<NA>'), 'cleaned_name'] + '_' + master_raw.loc[master_raw['match_key'].str.contains('<NA>'), 'mapped_ids'].astype(int).astype(str)
master_raw.loc[master_raw['match_key'].str.contains('<NA>')]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids


In [53]:
master_raw['match_key'] = master_raw['match_key'].str.replace('.0', '', regex=False)

In [54]:
master_raw['mapped_ids'] = master_raw['mapped_ids'].astype(int)

In [55]:
master_raw.head()

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
0,17030,3c76cab3-3df2-43dd-acaa-57e055bd32d0,24755.0,00-0040676,133244.0,12522.0,58203.0,4688380.0,NaN,NaN,3168422.0,WardCa00,NaN,16997.0,NaN,1730.0,41786.0,0.0,25323.0,NaN,Cam Ward,cam ward,QB,TEN,2002-05-25,23.4,2025.0,1.0,1.0,1.0,NaN,74.0,219.0,Miami (FL),2025,Cam Ward,Cam Ward_41786,41786
1,17031,270e09bc-8bf8-44b0-87ed-1fd014de4ab7,25968.0,00-0040668,131396.0,12524.0,58349.0,4432762.0,NaN,NaN,26710004.0,SandSh00,NaN,18479.0,NaN,1731.0,41930.0,0.0,26079.0,NaN,Shedeur Sanders,shedeur sanders,QB,CLE,2002-02-07,23.7,2025.0,5.0,6.0,144.0,NaN,74.0,212.0,Colorado,2025,Shedeur Sanders,Shedeur Sanders_41930,41930
2,17032,53848cba-bcad-4cd0-bb35-0a1f6ed111d1,23160.0,00-0040691,146409.0,12508.0,58227.0,4689114.0,NaN,NaN,26694019.0,DartJa00,NaN,18574.0,NaN,1732.0,41810.0,0.0,26082.0,NaN,Jaxson Dart,jaxson dart,QB,NYG,2003-05-13,22.4,2025.0,1.0,25.0,25.0,NaN,74.0,223.0,Ole Miss,2025,Jaxson Dart,Jaxson Dart_41810,41810
3,17033,b0eb5b67-4ef2-462d-bfe7-e73bcce1f6b0,22884.0,00-0040673,146590.0,12510.0,58297.0,4432734.0,NaN,NaN,26694943.0,MilrJa00,NaN,18477.0,NaN,1733.0,41882.0,0.0,25882.0,NaN,Jalen Milroe,jalen milroe,QB,SEA,2002-12-13,22.8,2025.0,3.0,28.0,92.0,NaN,74.0,216.0,Alabama,2025,Jalen Milroe,Jalen Milroe_41882,41882
4,17034,d89f0384-21a6-44e4-8aa6-324cb2103bf6,23680.0,00-0040234,127253.0,12500.0,58436.0,4889929.0,NaN,NaN,26746721.0,EwerQu00,NaN,18505.0,NaN,1734.0,42037.0,0.0,26048.0,NaN,Quinn Ewers,quinn ewers,QB,MIA,2003-05-13,22.4,2025.0,7.0,15.0,231.0,NaN,74.0,209.0,Texas,2025,Quinn Ewers,Quinn Ewers_42037,42037


In [56]:
master_raw.loc[(master_raw['cleaned_name'] == 'Travis Hunter'), 'match_key'] = 'Travis Hunter_99001'

In [58]:
master_raw[master_raw['cleaned_name'] == "Travis Hunter"]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
44,17074,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025,Travis Hunter,Travis Hunter_99001,41787


In [59]:
master_raw = master_raw.drop(master_raw.loc[(~master_raw['mapped_ids'].isin(yahoo_players['player_id'])) & (master_raw['cleaned_name'].isin(yahoo_players['cleaned_name'])) & (master_raw['cleaned_name'] != 'Travis Hunter')].index)

In [60]:
master_raw[master_raw['cleaned_name'] == "Travis Hunter"]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
44,17074,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025,Travis Hunter,Travis Hunter_99001,41787


In [62]:
master_raw.loc[(~master_raw['mapped_ids'].isin(yahoo_players['player_id'])) & (master_raw['cleaned_name'].isin(yahoo_players['cleaned_name'])) & (master_raw['cleaned_name'] == 'Travis Hunter')]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
44,17074,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025,Travis Hunter,Travis Hunter_99001,41787


In [63]:
master_raw.loc[(master_raw['mapped_ids'].isin(yahoo_players['player_id'])) & (~master_raw['cleaned_name'].isin(yahoo_players['cleaned_name']))]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
190,17220,80dce1ba-c997-47e6-a37e-e92fefcbdc1b,NaN,00-0040200,143934.0,12713.0,58387.0,4569923.0,NaN,NaN,26699665.0,BorrAn00,NaN,18743.0,NaN,NaN,41980.0,0.0,26106.0,NaN,Andres Borregales,andres borregales,PK,NEP,2002-01-02,23.8,2025.0,6.0,6.0,182.0,NaN,71.0,202.0,Miami (FL),2025,Andres Borregales,Andres Borregales_41980,41980
600,16833,fffee080-04ff-11ef-8a9a-5f3b29e3b4c8,26405.0,00-0039541,124169.0,11886.0,57660.0,4429153.0,41163.0,NaN,3171022.0,GrifLi00,NaN,17973.0,NaN,NaN,41163.0,0.0,NaN,1231735.0,Lideatrick Griffin,lideatrick griffin,WR,FA,2001-01-23,24.7,2024.0,NaN,NaN,NaN,NaN,70.0,181.0,Mississippi State,2025,Lideatrick Griffin,Lideatrick Griffin_41163,41163
754,16208,3a00b600-ea03-11ed-8d34-41182339b277,22941.0,NaN,NaN,9759.0,NaN,NaN,NaN,NaN,3123939.0,NaN,NaN,16823.0,NaN,NaN,40461.0,0.0,NaN,1172798.0,CJ Johnson,cj johnson,WR,FA,2000-11-04,23.3,2023.0,NaN,NaN,NaN,NaN,73.0,225.0,East Carolina,2025,CJ Johnson,CJ Johnson_40461,40461
886,16342,962496d4-9ffa-471b-8dd2-fc5e60fee84b,25304.0,00-0038621,104155.0,9501.0,56075.0,4427095.0,40222.0,NaN,3126241.0,DougDe00,NaN,16751.0,NaN,1468.0,40222.0,0.0,24180.0,1175693.0,Demario Douglas,demario douglas,WR,NEP,2000-12-08,24.9,2023.0,6.0,33.0,210.0,NaN,68.0,192.0,Liberty,2025,Demario Douglas,Demario Douglas_40222,40222
918,16374,3b590cad-60c1-471d-a54a-d593a676afa9,25470.0,00-0038507,111276.0,11024.0,56121.0,4879650.0,40251.0,NaN,2672034.0,DuboGr00,NaN,16852.0,NaN,NaN,40251.0,0.0,NaN,1323751.0,Grant Dubose,grant dubose,WR,FA,2001-06-20,24.3,2023.0,7.0,39.0,256.0,NaN,74.0,201.0,Charlotte,2025,Grant Dubose,Grant Dubose_40251,40251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6795,9200,9d404288-65c5-414f-8ea5-ceb97eccaea0,9446.0,00-0026293,4467.0,312.0,4487.0,11387.0,8930.0,NaN,1615610.0,SlatMa00,matt-slater-1,5786.0,4904.0,NaN,8930.0,0.0,7651.0,234403.0,Matt Slater,matthew slater,WR,FA,1985-09-09,39.5,2008.0,5.0,18.0,153.0,NaN,72.0,205.0,UCLA,2025,Matt Slater,Matt Slater_8930,8930
6855,9269,c1b5db36-a2e2-4bbc-87c1-cdd216b4474c,9577.0,00-0026364,4538.0,108.0,768.0,11458.0,9001.0,NaN,1116922.0,JohnSt00,steve-johnson-2,5798.0,4962.0,NaN,9001.0,333980.0,2950.0,NaN,Stevie Johnson,stevie johnson,WR,FA,1986-07-22,34.8,2008.0,7.0,17.0,224.0,StevieJohnson13,74.0,207.0,Kentucky,2025,Stevie Johnson,Stevie Johnson_9001,9001
6890,9319,40cda44b-2ee3-4ad1-834e-995e30db84d4,9581.0,00-0025944,4655.0,775.0,2507374.0,11923.0,9066.0,NaN,1616746.0,HausSt00,NaN,5935.0,5030.0,NaN,9066.0,406186.0,12594.0,NaN,Steven Hauschka,steven hauschka,PK,FA,1985-06-29,36.7,2008.0,NaN,NaN,NaN,NaN,76.0,210.0,North Carolina State,2025,Steven Hauschka,Steven Hauschka_9066,9066
7864,7422,d81844de-54c3-42ee-9850-072dc4131b6f,9040.0,00-0022943,1753.0,663.0,2506122.0,5557.0,6791.0,NaN,493107.0,WatsBe00,benjamin-watson-1,3872.0,48.0,NaN,6791.0,246082.0,11756.0,NaN,Ben Watson,ben watson,TE,FA,1980-12-18,40.4,2004.0,1.0,32.0,32.0,BenjaminSWatson,75.0,255.0,Georgia,2025,Ben Watson,Ben Watson_6791,6791


In [64]:
master_raw[master_raw['mapped_ids'].duplicated(keep=False)]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
1100,15717,5d4ef6da-a6b5-4d9c-adc0-28dfdbe9315e,22947.0,00-0037258,83659.0,8129.0,54572.0,4360238.0,34063.0,NaN,2964236.0,PierDa01,dameon-pierce-1,15875.0,NaN,1207.0,34063.0,0.0,23223.0,1108998.0,Dameon Pierce,dameon pierce,RB,HOU,2000-02-19,25.7,2022.0,4.0,2.0,107.0,NaN,70.0,215.0,Florida,2025,Dameon Pierce,Dameon Pierce_34063,34063
1142,15759,2995df34-ee81-4320-be1e-1c497b83c927,22895.0,00-0037614,97400.0,8147.0,54509.0,4567096.0,33999.0,NaN,3121171.0,MetcJo00,john-metchie-1,16069.0,NaN,NaN,34063.0,0.0,23160.0,1160348.0,John Metchie,john metchie,WR,PHI,2000-07-18,25.2,2022.0,2.0,12.0,44.0,NaN,72.0,195.0,Alabama,2025,John Metchie,John Metchie_34063,34063
2297,14117,064c4eda-1b10-40ac-a9d2-66caf76a213a,18804.0,00-0035457,34427.0,6063.0,48601.0,3894912.0,32511.0,NaN,2180626.0,JohnTy03,NaN,13615.0,14690.0,NaN,32511.0,865567.0,20838.0,865567.0,Tyron Johnson,tyron johnson,WR,FA,1996-01-08,28.1,2019.0,NaN,NaN,NaN,NaN,73.0,193.0,Oklahoma State,2025,Tyron Johnson,Tyron Johnson_32511,32511
2679,16859,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2180626.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Tyron Billy-Johnson,tyron billy-johnson,XX,FA,1996-01-08,29.1,2019.0,NaN,NaN,NaN,NaN,73.0,193.0,Oklahoma State,2025,Tyron Billy-Johnson,Tyron Billy-Johnson_32511,32511


In [65]:
master_raw.loc[master_raw['cleaned_name'] == 'John Metchie', ['stats_id', 'mapped_ids', 'match_key']] = [33999, 33999, 'John_Metchie_33999']

In [66]:
master_raw[master_raw['mapped_ids'].duplicated(keep=False)]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
2297,14117,064c4eda-1b10-40ac-a9d2-66caf76a213a,18804.0,00-0035457,34427.0,6063.0,48601.0,3894912.0,32511.0,NaN,2180626.0,JohnTy03,NaN,13615.0,14690.0,NaN,32511.0,865567.0,20838.0,865567.0,Tyron Johnson,tyron johnson,WR,FA,1996-01-08,28.1,2019.0,NaN,NaN,NaN,NaN,73.0,193.0,Oklahoma State,2025,Tyron Johnson,Tyron Johnson_32511,32511
2679,16859,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2180626.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Tyron Billy-Johnson,tyron billy-johnson,XX,FA,1996-01-08,29.1,2019.0,NaN,NaN,NaN,NaN,73.0,193.0,Oklahoma State,2025,Tyron Billy-Johnson,Tyron Billy-Johnson_32511,32511


In [67]:
master_raw.loc[~(master_raw['match_key'].isin(yahoo_players['match_key']))]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
190,17220,80dce1ba-c997-47e6-a37e-e92fefcbdc1b,NaN,00-0040200,143934.0,12713.0,58387.0,4569923.0,NaN,NaN,26699665.0,BorrAn00,NaN,18743.0,NaN,NaN,41980.0,0.0,26106.0,NaN,Andres Borregales,andres borregales,PK,NEP,2002-01-02,23.8,2025.0,6.0,6.0,182.0,NaN,71.0,202.0,Miami (FL),2025,Andres Borregales,Andres Borregales_41980,41980
358,16590,ec7b8cf0-0bb9-11ef-bb45-99966e93bb71,23002.0,NaN,NaN,12259.0,NaN,NaN,NaN,NaN,3123935.0,NaN,NaN,16999.0,NaN,NaN,41559.0,0.0,NaN,NaN,Taulia Tagovailoa,taulia tagovailoa,QB,FA,2000-02-15,25.7,2024.0,NaN,NaN,NaN,NaN,71.0,200.0,Maryland,2025,Taulia Tagovailoa,Taulia Tagovailoa_41559,41559
585,16818,8e6fc170-07d2-11ef-95d0-7db2ea91136a,NaN,NaN,NaN,12188.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41477.0,0.0,NaN,NaN,Justin Strong,justin strong,RB,FA,NaN,NaN,2024.0,NaN,NaN,NaN,NaN,72.0,205.0,Southern Illinois,2025,Justin Strong,Justin Strong_41477,41477
600,16833,fffee080-04ff-11ef-8a9a-5f3b29e3b4c8,26405.0,00-0039541,124169.0,11886.0,57660.0,4429153.0,41163.0,NaN,3171022.0,GrifLi00,NaN,17973.0,NaN,NaN,41163.0,0.0,NaN,1231735.0,Lideatrick Griffin,lideatrick griffin,WR,FA,2001-01-23,24.7,2024.0,NaN,NaN,NaN,NaN,70.0,181.0,Mississippi State,2025,Lideatrick Griffin,Lideatrick Griffin_41163,41163
754,16208,3a00b600-ea03-11ed-8d34-41182339b277,22941.0,NaN,NaN,9759.0,NaN,NaN,NaN,NaN,3123939.0,NaN,NaN,16823.0,NaN,NaN,40461.0,0.0,NaN,1172798.0,CJ Johnson,cj johnson,WR,FA,2000-11-04,23.3,2023.0,NaN,NaN,NaN,NaN,73.0,225.0,East Carolina,2025,CJ Johnson,CJ Johnson_40461,40461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12034,9793,20af511b-03b7-4905-bc41-158f286b99c6,NaN,NaN,NaN,2864.0,2553937.0,12899.0,29138.0,NaN,2175732.0,NaN,NaN,6002.0,5243.0,NaN,9736.0,245814.0,17332.0,NaN,Mike Reilly,mike reilly,QB,STL,1985-01-25,36.3,0.0,NaN,NaN,NaN,NaN,76.0,240.0,Central Washington,2025,Mike Reilly,Mike Reilly_9736,9736
12036,9796,NaN,NaN,00-0026792,NaN,NaN,NaN,13027.0,NaN,NaN,1675238.0,WillPa01,NaN,6401.0,5595.0,NaN,9721.0,267416.0,NaN,NaN,Patrick Williams,patrick williams,WR,BAL,1986-01-13,35.3,0.0,NaN,NaN,NaN,NaN,73.0,204.0,Colorado,2025,Patrick Williams,Patrick Williams_9721,9721
12038,9800,NaN,NaN,00-0024825,NaN,NaN,NaN,11142.0,NaN,NaN,1226291.0,NordJa00,NaN,5467.0,4608.0,NaN,8692.0,219960.0,NaN,NaN,Jake Nordin,jake nordin,TE,DET,1984-07-08,36.8,0.0,NaN,NaN,NaN,NaN,75.0,255.0,Northern Illinois,2025,Jake Nordin,Jake Nordin_8692,8692
12039,9806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6202.0,5498.0,NaN,9122.0,463275.0,NaN,NaN,Sam Giguere,sam giguere,WR,FA,1985-07-11,35.8,0.0,NaN,NaN,NaN,NaN,71.0,215.0,Sherbrooke,2025,Sam Giguere,Sam Giguere_9122,9122


In [ ]:
master_raw[master_raw['cleaned_name'] == "Travis Hunter"]

In [68]:
master_raw.loc[(master_raw['mapped_ids'].isin(yahoo_players['player_id'])) & (~master_raw['cleaned_name'].isin(yahoo_players['cleaned_name']))]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
190,17220,80dce1ba-c997-47e6-a37e-e92fefcbdc1b,NaN,00-0040200,143934.0,12713.0,58387.0,4569923.0,NaN,NaN,26699665.0,BorrAn00,NaN,18743.0,NaN,NaN,41980.0,0.0,26106.0,NaN,Andres Borregales,andres borregales,PK,NEP,2002-01-02,23.8,2025.0,6.0,6.0,182.0,NaN,71.0,202.0,Miami (FL),2025,Andres Borregales,Andres Borregales_41980,41980
600,16833,fffee080-04ff-11ef-8a9a-5f3b29e3b4c8,26405.0,00-0039541,124169.0,11886.0,57660.0,4429153.0,41163.0,NaN,3171022.0,GrifLi00,NaN,17973.0,NaN,NaN,41163.0,0.0,NaN,1231735.0,Lideatrick Griffin,lideatrick griffin,WR,FA,2001-01-23,24.7,2024.0,NaN,NaN,NaN,NaN,70.0,181.0,Mississippi State,2025,Lideatrick Griffin,Lideatrick Griffin_41163,41163
754,16208,3a00b600-ea03-11ed-8d34-41182339b277,22941.0,NaN,NaN,9759.0,NaN,NaN,NaN,NaN,3123939.0,NaN,NaN,16823.0,NaN,NaN,40461.0,0.0,NaN,1172798.0,CJ Johnson,cj johnson,WR,FA,2000-11-04,23.3,2023.0,NaN,NaN,NaN,NaN,73.0,225.0,East Carolina,2025,CJ Johnson,CJ Johnson_40461,40461
886,16342,962496d4-9ffa-471b-8dd2-fc5e60fee84b,25304.0,00-0038621,104155.0,9501.0,56075.0,4427095.0,40222.0,NaN,3126241.0,DougDe00,NaN,16751.0,NaN,1468.0,40222.0,0.0,24180.0,1175693.0,Demario Douglas,demario douglas,WR,NEP,2000-12-08,24.9,2023.0,6.0,33.0,210.0,NaN,68.0,192.0,Liberty,2025,Demario Douglas,Demario Douglas_40222,40222
918,16374,3b590cad-60c1-471d-a54a-d593a676afa9,25470.0,00-0038507,111276.0,11024.0,56121.0,4879650.0,40251.0,NaN,2672034.0,DuboGr00,NaN,16852.0,NaN,NaN,40251.0,0.0,NaN,1323751.0,Grant Dubose,grant dubose,WR,FA,2001-06-20,24.3,2023.0,7.0,39.0,256.0,NaN,74.0,201.0,Charlotte,2025,Grant Dubose,Grant Dubose_40251,40251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6795,9200,9d404288-65c5-414f-8ea5-ceb97eccaea0,9446.0,00-0026293,4467.0,312.0,4487.0,11387.0,8930.0,NaN,1615610.0,SlatMa00,matt-slater-1,5786.0,4904.0,NaN,8930.0,0.0,7651.0,234403.0,Matt Slater,matthew slater,WR,FA,1985-09-09,39.5,2008.0,5.0,18.0,153.0,NaN,72.0,205.0,UCLA,2025,Matt Slater,Matt Slater_8930,8930
6855,9269,c1b5db36-a2e2-4bbc-87c1-cdd216b4474c,9577.0,00-0026364,4538.0,108.0,768.0,11458.0,9001.0,NaN,1116922.0,JohnSt00,steve-johnson-2,5798.0,4962.0,NaN,9001.0,333980.0,2950.0,NaN,Stevie Johnson,stevie johnson,WR,FA,1986-07-22,34.8,2008.0,7.0,17.0,224.0,StevieJohnson13,74.0,207.0,Kentucky,2025,Stevie Johnson,Stevie Johnson_9001,9001
6890,9319,40cda44b-2ee3-4ad1-834e-995e30db84d4,9581.0,00-0025944,4655.0,775.0,2507374.0,11923.0,9066.0,NaN,1616746.0,HausSt00,NaN,5935.0,5030.0,NaN,9066.0,406186.0,12594.0,NaN,Steven Hauschka,steven hauschka,PK,FA,1985-06-29,36.7,2008.0,NaN,NaN,NaN,NaN,76.0,210.0,North Carolina State,2025,Steven Hauschka,Steven Hauschka_9066,9066
7864,7422,d81844de-54c3-42ee-9850-072dc4131b6f,9040.0,00-0022943,1753.0,663.0,2506122.0,5557.0,6791.0,NaN,493107.0,WatsBe00,benjamin-watson-1,3872.0,48.0,NaN,6791.0,246082.0,11756.0,NaN,Ben Watson,ben watson,TE,FA,1980-12-18,40.4,2004.0,1.0,32.0,32.0,BenjaminSWatson,75.0,255.0,Georgia,2025,Ben Watson,Ben Watson_6791,6791


In [ ]:
master_raw[master_raw['cleaned_name'] == "Travis Hunter"]

In [69]:
master_raw['last_name'] = master_raw['cleaned_name'].str.split().str[-1]
master_raw.head()

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids,last_name
0,17030,3c76cab3-3df2-43dd-acaa-57e055bd32d0,24755.0,00-0040676,133244.0,12522.0,58203.0,4688380.0,NaN,NaN,3168422.0,WardCa00,NaN,16997.0,NaN,1730.0,41786.0,0.0,25323.0,NaN,Cam Ward,cam ward,QB,TEN,2002-05-25,23.4,2025.0,1.0,1.0,1.0,NaN,74.0,219.0,Miami (FL),2025,Cam Ward,Cam Ward_41786,41786,Ward
1,17031,270e09bc-8bf8-44b0-87ed-1fd014de4ab7,25968.0,00-0040668,131396.0,12524.0,58349.0,4432762.0,NaN,NaN,26710004.0,SandSh00,NaN,18479.0,NaN,1731.0,41930.0,0.0,26079.0,NaN,Shedeur Sanders,shedeur sanders,QB,CLE,2002-02-07,23.7,2025.0,5.0,6.0,144.0,NaN,74.0,212.0,Colorado,2025,Shedeur Sanders,Shedeur Sanders_41930,41930,Sanders
2,17032,53848cba-bcad-4cd0-bb35-0a1f6ed111d1,23160.0,00-0040691,146409.0,12508.0,58227.0,4689114.0,NaN,NaN,26694019.0,DartJa00,NaN,18574.0,NaN,1732.0,41810.0,0.0,26082.0,NaN,Jaxson Dart,jaxson dart,QB,NYG,2003-05-13,22.4,2025.0,1.0,25.0,25.0,NaN,74.0,223.0,Ole Miss,2025,Jaxson Dart,Jaxson Dart_41810,41810,Dart
3,17033,b0eb5b67-4ef2-462d-bfe7-e73bcce1f6b0,22884.0,00-0040673,146590.0,12510.0,58297.0,4432734.0,NaN,NaN,26694943.0,MilrJa00,NaN,18477.0,NaN,1733.0,41882.0,0.0,25882.0,NaN,Jalen Milroe,jalen milroe,QB,SEA,2002-12-13,22.8,2025.0,3.0,28.0,92.0,NaN,74.0,216.0,Alabama,2025,Jalen Milroe,Jalen Milroe_41882,41882,Milroe
4,17034,d89f0384-21a6-44e4-8aa6-324cb2103bf6,23680.0,00-0040234,127253.0,12500.0,58436.0,4889929.0,NaN,NaN,26746721.0,EwerQu00,NaN,18505.0,NaN,1734.0,42037.0,0.0,26048.0,NaN,Quinn Ewers,quinn ewers,QB,MIA,2003-05-13,22.4,2025.0,7.0,15.0,231.0,NaN,74.0,209.0,Texas,2025,Quinn Ewers,Quinn Ewers_42037,42037,Ewers


In [ ]:
master_raw[master_raw['cleaned_name'] == "Travis Hunter"]

In [70]:
yahoo_players['last_name'] = yahoo_players['cleaned_name'].str.split().str[-1]
yahoo_players.head()

,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season,cleaned_name,match_key,last_name
0,10,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,331.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2014,Adam Vinatieri,Adam Vinatieri_3727,Vinatieri
1,9,TE,nfl.p.3950,Atl,Atlanta Falcons,nfl.t.1,https://sports.yahoo.com/nfl/teams/atlanta/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/1_ZQRTBuF_fZ...,https://s.yimg.com/iu/api/res/1.2/1_ZQRTBuF_fZ...,False,False,False,0,Tony,Tony,Tony Gonzalez,Gonzalez,3950,331.p.3950,O,TE,https://sports.yahoo.com/nfl/players/3950,2014,Tony Gonzalez,Tony Gonzalez_3950,Gonzalez
2,4,QB,nfl.p.4256,Den,Denver Broncos,nfl.t.7,https://sports.yahoo.com/nfl/teams/denver/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/UCWtIm9kEbbH...,https://s.yimg.com/iu/api/res/1.2/UCWtIm9kEbbH...,False,False,False,0,Peyton,Peyton,Peyton Manning,Manning,4256,331.p.4256,O,QB,https://sports.yahoo.com/nfl/players/4256,2014,Peyton Manning,Peyton Manning_4256,Manning
3,4,K,nfl.p.4269,Ari,Arizona Cardinals,nfl.t.22,https://sports.yahoo.com/nfl/teams/arizona/,['K'],small,https://s.yimg.com/iu/api/res/1.2/z4ki.EZ6BaZp...,https://s.yimg.com/iu/api/res/1.2/z4ki.EZ6BaZp...,False,False,False,0,Phil,Phil,Phil Dawson,Dawson,4269,331.p.4269,K,K,https://sports.yahoo.com/nfl/players/4269,2014,Phil Dawson,Phil Dawson_4269,Dawson
4,10,QB,nfl.p.4416,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/YdWdJ6P7MDCj...,https://s.yimg.com/iu/api/res/1.2/YdWdJ6P7MDCj...,False,False,False,0,Matt,Matt,Matt Hasselbeck,Hasselbeck,4416,331.p.4416,O,QB,https://sports.yahoo.com/nfl/players/4416,2014,Matt Hasselbeck,Matt Hasselbeck_4416,Hasselbeck


In [ ]:
master_raw[master_raw['cleaned_name'] == "Travis Hunter"]

In [71]:
master_raw.loc[(master_raw['mapped_ids'].isin(yahoo_players['player_id'])) & (~master_raw['cleaned_name'].isin(yahoo_players['cleaned_name'])) & (~master_raw['last_name'].isin(yahoo_players['last_name']))]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids,last_name
918,16374,3b590cad-60c1-471d-a54a-d593a676afa9,25470.0,00-0038507,111276.0,11024.0,56121.0,4879650.0,40251.0,NaN,2672034.0,DuboGr00,NaN,16852.0,NaN,NaN,40251.0,0.0,NaN,1323751.0,Grant Dubose,grant dubose,WR,FA,2001-06-20,24.3,2023.0,7.0,39.0,256.0,NaN,74.0,201.0,Charlotte,2025,Grant Dubose,Grant Dubose_40251,40251,Dubose
1812,15677,7656034d-8647-49e5-b0d4-b708423042ee,23339.0,00-0036696,NaN,7903.0,NaN,NaN,33837.0,NaN,2247169.0,NaN,NaN,15561.0,NaN,NaN,NaN,0.0,NaN,921082.0,Briley Moore-McKinney,briley moore-mckinney,TE,FA,1998-01-13,24.1,2021.0,NaN,NaN,NaN,NaN,76.0,250.0,Kansas State,2025,Briley Moore-McKinney,Briley Moore-McKinney_33837,33837,Moore-McKinney
3720,12809,d7fe9224-7bd7-40e5-a314-1152ed5334d5,NaN,00-0032341,10814.0,3336.0,43469.0,4002672.0,29414.0,NaN,2235403.0,BoehMo00,NaN,11212.0,11470.0,NaN,29414.0,914919.0,18097.0,NaN,Moritz Boehringer,moritz boehringer,WR,FA,1993-10-16,27.6,2016.0,6.0,5.0,180.0,MoBoehringer,77.0,250.0,Germany,2025,Moritz Boehringer,Moritz Boehringer_29414,29414,Boehringer


In [72]:
master_raw[['last_name', 'cleaned_name', 'match_key']] = master_raw[['last_name', 'cleaned_name', 'match_key']].apply(lambda x: x.str.lower())
master_raw.head()

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids,last_name
0,17030,3c76cab3-3df2-43dd-acaa-57e055bd32d0,24755.0,00-0040676,133244.0,12522.0,58203.0,4688380.0,NaN,NaN,3168422.0,WardCa00,NaN,16997.0,NaN,1730.0,41786.0,0.0,25323.0,NaN,Cam Ward,cam ward,QB,TEN,2002-05-25,23.4,2025.0,1.0,1.0,1.0,NaN,74.0,219.0,Miami (FL),2025,cam ward,cam ward_41786,41786,ward
1,17031,270e09bc-8bf8-44b0-87ed-1fd014de4ab7,25968.0,00-0040668,131396.0,12524.0,58349.0,4432762.0,NaN,NaN,26710004.0,SandSh00,NaN,18479.0,NaN,1731.0,41930.0,0.0,26079.0,NaN,Shedeur Sanders,shedeur sanders,QB,CLE,2002-02-07,23.7,2025.0,5.0,6.0,144.0,NaN,74.0,212.0,Colorado,2025,shedeur sanders,shedeur sanders_41930,41930,sanders
2,17032,53848cba-bcad-4cd0-bb35-0a1f6ed111d1,23160.0,00-0040691,146409.0,12508.0,58227.0,4689114.0,NaN,NaN,26694019.0,DartJa00,NaN,18574.0,NaN,1732.0,41810.0,0.0,26082.0,NaN,Jaxson Dart,jaxson dart,QB,NYG,2003-05-13,22.4,2025.0,1.0,25.0,25.0,NaN,74.0,223.0,Ole Miss,2025,jaxson dart,jaxson dart_41810,41810,dart
3,17033,b0eb5b67-4ef2-462d-bfe7-e73bcce1f6b0,22884.0,00-0040673,146590.0,12510.0,58297.0,4432734.0,NaN,NaN,26694943.0,MilrJa00,NaN,18477.0,NaN,1733.0,41882.0,0.0,25882.0,NaN,Jalen Milroe,jalen milroe,QB,SEA,2002-12-13,22.8,2025.0,3.0,28.0,92.0,NaN,74.0,216.0,Alabama,2025,jalen milroe,jalen milroe_41882,41882,milroe
4,17034,d89f0384-21a6-44e4-8aa6-324cb2103bf6,23680.0,00-0040234,127253.0,12500.0,58436.0,4889929.0,NaN,NaN,26746721.0,EwerQu00,NaN,18505.0,NaN,1734.0,42037.0,0.0,26048.0,NaN,Quinn Ewers,quinn ewers,QB,MIA,2003-05-13,22.4,2025.0,7.0,15.0,231.0,NaN,74.0,209.0,Texas,2025,quinn ewers,quinn ewers_42037,42037,ewers


In [73]:
yahoo_players[['last_name', 'cleaned_name', 'match_key']] = yahoo_players[['last_name', 'cleaned_name', 'match_key']].apply(lambda x: x.str.lower())

In [74]:
master_raw.loc[(master_raw['mapped_ids'].isin(yahoo_players['player_id'])) & (~master_raw['cleaned_name'].isin(yahoo_players['cleaned_name'])) & (~master_raw['last_name'].isin(yahoo_players['last_name']))]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids,last_name
1812,15677,7656034d-8647-49e5-b0d4-b708423042ee,23339.0,00-0036696,NaN,7903.0,NaN,NaN,33837.0,NaN,2247169.0,NaN,NaN,15561.0,NaN,NaN,NaN,0.0,NaN,921082.0,Briley Moore-McKinney,briley moore-mckinney,TE,FA,1998-01-13,24.1,2021.0,NaN,NaN,NaN,NaN,76.0,250.0,Kansas State,2025,briley moore-mckinney,briley moore-mckinney_33837,33837,moore-mckinney
3720,12809,d7fe9224-7bd7-40e5-a314-1152ed5334d5,NaN,00-0032341,10814.0,3336.0,43469.0,4002672.0,29414.0,NaN,2235403.0,BoehMo00,NaN,11212.0,11470.0,NaN,29414.0,914919.0,18097.0,NaN,Moritz Boehringer,moritz boehringer,WR,FA,1993-10-16,27.6,2016.0,6.0,5.0,180.0,MoBoehringer,77.0,250.0,Germany,2025,moritz boehringer,moritz boehringer_29414,29414,boehringer


In [75]:
master_raw.loc[master_raw['cleaned_name'] == 'briley moore-mckinney', ['cleaned_name', 'match_key', 'last_name']] = ['briley moore', 'briley moore_338337', 'moore']
master_raw.loc[master_raw['cleaned_name'] == 'moritz boehringer', ['cleaned_name', 'match_key', 'last_name']] = ['moritz bohringer', 'moritz bohringer_29414', 'bohringer']

In [76]:
master_raw.loc[(master_raw['mapped_ids'].isin(yahoo_players['player_id'])) & (~master_raw['cleaned_name'].isin(yahoo_players['cleaned_name'])) & (~master_raw['last_name'].isin(yahoo_players['last_name']))]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids,last_name


In [77]:
master_raw[master_raw['stats_id'].isna()]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids,last_name
59,17089,483872b0-2630-11f0-b7f1-df7c830143e7,27218.0,NaN,NaN,12460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18559.0,NaN,NaN,NaN,0.0,26081.0,NaN,Antwane Wells,antwane wells,WR,NYG,2001-04-02,24.5,2025.0,NaN,NaN,NaN,NaN,72.0,205.0,Ole Miss,2025,antwane wells,antwane wells_42444,42444,wells
141,17171,1e95c7fc-ecf7-4fcb-82d8-d77e2b6872d5,27315.0,00-0040705,129918.0,12540.0,58308.0,4431268.0,NaN,NaN,3162452.0,DikeCh00,NaN,18641.0,NaN,1814.0,NaN,0.0,25987.0,NaN,Chimere Dike,chimere dike,WR,TEN,2001-12-14,23.8,2025.0,4.0,1.0,103.0,NaN,73.0,196.0,Florida,2025,chimere dike,chimere dike_41903,41903,dike
245,17277,7bf5a2e0-2512-11f0-9825-33176da47896,27261.0,00-0040279,99867.0,12961.0,58715.0,4568263.0,NaN,NaN,NaN,FitzRy01,NaN,18508.0,NaN,NaN,NaN,0.0,26108.0,NaN,Ryan Fitzgerald,ryan fitzgerald,PK,CAR,2000-08-13,25.2,2025.0,NaN,NaN,NaN,NaN,71.0,193.0,Florida State,2025,ryan fitzgerald,ryan fitzgerald_42262,42262,fitzgerald
344,17396,NaN,NaN,00-0040534,123406.0,NaN,58980.0,4570738.0,NaN,NaN,NaN,WillKe08,NaN,18841.0,NaN,NaN,NaN,0.0,NaN,NaN,Ke'Shawn Williams,keshawn williams,WR,PIT,2001-12-27,23.8,2025.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,keshawn williams,keshawn williams_42278,42278,williams
402,16634,ffd59710-0582-11ef-823c-c3942ac17525,23957.0,00-0039521,101130.0,11921.0,57639.0,4428811.0,41202.0,NaN,3131562.0,WeavXa00,NaN,18041.0,NaN,NaN,NaN,0.0,25032.0,1180538.0,Xavier Weaver,xavier weaver,WR,ARI,2000-10-12,25.0,2024.0,NaN,NaN,NaN,NaN,73.0,185.0,Colorado,2025,xavier weaver,xavier weaver_41202,41202,weaver
406,16638,NaN,NaN,NaN,NaN,NaN,NaN,4427225.0,NaN,NaN,NaN,NaN,NaN,17006.0,NaN,NaN,NaN,0.0,NaN,NaN,Zakhari Franklin,zakhari franklin,WR,FA,NaN,NaN,2024.0,NaN,NaN,NaN,NaN,NaN,NaN,Mississippi,2025,zakhari franklin,zakhari franklin_42479,42479,franklin
741,16195,ea3d1393-8c3b-4e81-a519-f636d28d692e,24354.0,00-0038393,99711.0,9486.0,56024.0,4428850.0,40201.0,NaN,3122375.0,WickDo00,dontayvion-wicks-1,16658.0,NaN,1448.0,NaN,0.0,23253.0,1172428.0,Dontayvion Wicks,dontayvion wicks,WR,GBP,2001-06-16,24.3,2023.0,5.0,24.0,159.0,NaN,73.0,206.0,Virginia,2025,dontayvion wicks,dontayvion wicks_40201,40201,wicks
766,16220,5d4a5e4f-7c74-4b4c-b8e5-33bb831e3057,23821.0,00-0038394,77888.0,10220.0,56027.0,4362523.0,40180.0,NaN,2962073.0,MallWi00,will-mallory-1,17014.0,NaN,NaN,NaN,0.0,23143.0,1121873.0,Will Mallory,will mallory,TE,IND,1999-06-22,26.3,2023.0,5.0,27.0,162.0,NaN,76.0,239.0,Miami (FL),2025,will mallory,will mallory_40180,40180,mallory
1812,15677,7656034d-8647-49e5-b0d4-b708423042ee,23339.0,00-0036696,NaN,7903.0,NaN,NaN,33837.0,NaN,2247169.0,NaN,NaN,15561.0,NaN,NaN,NaN,0.0,NaN,921082.0,Briley Moore-McKinney,briley moore-mckinney,TE,FA,1998-01-13,24.1,2021.0,NaN,NaN,NaN,NaN,76.0,250.0,Kansas State,2025,briley moore,briley moore_338337,33837,moore
2271,14091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Tre Watson,tre watson,RB,FA,NaN,NaN,2019.0,NaN,NaN,NaN,NaN,71.0,195.0,Texas,2025,tre watson,tre watson_42430,42430,watson


In [79]:
master_raw[master_raw['cleaned_name'] == "travis hunter"]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids,last_name
44,17074,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025,travis hunter,travis hunter_99001,41787,hunter


In [80]:
master_raw[master_raw['cleaned_name'] == 'tony carter']

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids,last_name
10901,1858,1e436b1f-6706-45be-b1b6-ce2969df48fe,NaN,00-0026515,5191.0,780.0,34722.0,12771.0,9608.0,NaN,1674227.0,CartTo99,NaN,112.0,NaN,NaN,NaN,284092.0,12636.0,NaN,Tony Carter,tony carter,RB,FA,1972-08-23,48.7,0.0,NaN,NaN,NaN,NaN,69.0,175.0,Minnesota,2025,tony carter,tony carter_9608,9608,carter


In [81]:
master_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4331 entries, 0 to 12040
Data columns (total 39 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   mfl_id            4331 non-null   int64  
 1   sportradar_id     3254 non-null   object 
 2   fantasypros_id    2487 non-null   float64
 3   gsis_id           3329 non-null   object 
 4   pff_id            3136 non-null   float64
 5   sleeper_id        2834 non-null   float64
 6   nfl_id            3081 non-null   float64
 7   espn_id           3445 non-null   float64
 8   yahoo_id          2680 non-null   float64
 9   fleaflicker_id    489 non-null    float64
 10  cbs_id            4078 non-null   float64
 11  pfr_id            3521 non-null   object 
 12  cfbref_id         1830 non-null   object 
 13  rotowire_id       4221 non-null   float64
 14  rotoworld_id      3179 non-null   float64
 15  ktc_id            394 non-null    float64
 16  stats_id          4314 non-null   float64
 17 

In [82]:
master_raw = master_raw.loc[master_raw['cleaned_name'] != 'tony carter']

In [83]:
master_raw[master_raw['cleaned_name'] == "travis hunter"]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids,last_name
44,17074,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025,travis hunter,travis hunter_99001,41787,hunter


In [84]:
master_raw.loc[(master_raw['stats_id'].isna()), 'stats_id'] = master_raw.loc[(master_raw['stats_id'].isna()), 'mapped_ids']

In [85]:
master_raw['stats_id'].isna().sum()

np.int64(0)

In [86]:
master_raw[master_raw['cleaned_name'] == "travis hunter"]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids,last_name
44,17074,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025,travis hunter,travis hunter_99001,41787,hunter


In [87]:
master_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4330 entries, 0 to 12040
Data columns (total 39 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   mfl_id            4330 non-null   int64  
 1   sportradar_id     3253 non-null   object 
 2   fantasypros_id    2487 non-null   float64
 3   gsis_id           3328 non-null   object 
 4   pff_id            3135 non-null   float64
 5   sleeper_id        2833 non-null   float64
 6   nfl_id            3080 non-null   float64
 7   espn_id           3444 non-null   float64
 8   yahoo_id          2679 non-null   float64
 9   fleaflicker_id    489 non-null    float64
 10  cbs_id            4077 non-null   float64
 11  pfr_id            3520 non-null   object 
 12  cfbref_id         1830 non-null   object 
 13  rotowire_id       4220 non-null   float64
 14  rotoworld_id      3179 non-null   float64
 15  ktc_id            394 non-null    float64
 16  stats_id          4330 non-null   float64
 17 

In [88]:
master_raw[['stats_id', 'mapped_ids', 'match_key']].nunique()

stats_id      4329
mapped_ids    4329
match_key     4330
dtype: int64

In [89]:
master_raw[
    master_raw['stats_id'].duplicated(keep=False) | 
    master_raw['mapped_ids'].duplicated(keep=False) | 
    master_raw['match_key'].duplicated(keep=False)
]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids,last_name
2297,14117,064c4eda-1b10-40ac-a9d2-66caf76a213a,18804.0,00-0035457,34427.0,6063.0,48601.0,3894912.0,32511.0,NaN,2180626.0,JohnTy03,NaN,13615.0,14690.0,NaN,32511.0,865567.0,20838.0,865567.0,Tyron Johnson,tyron johnson,WR,FA,1996-01-08,28.1,2019.0,NaN,NaN,NaN,NaN,73.0,193.0,Oklahoma State,2025,tyron johnson,tyron johnson_32511,32511,johnson
2679,16859,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2180626.0,NaN,NaN,NaN,NaN,NaN,32511.0,0.0,NaN,NaN,Tyron Billy-Johnson,tyron billy-johnson,XX,FA,1996-01-08,29.1,2019.0,NaN,NaN,NaN,NaN,73.0,193.0,Oklahoma State,2025,tyron billy-johnson,tyron billy-johnson_32511,32511,billy-johnson


In [90]:
master_raw.loc[(master_raw['stats_id'].duplicated(keep=False)), 'stats_id'] = master_raw.loc[(master_raw['stats_id'].duplicated(keep=False)), 'mapped_ids']

master_raw[
    master_raw['mapped_ids'].duplicated(keep=False)
]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids,last_name
2297,14117,064c4eda-1b10-40ac-a9d2-66caf76a213a,18804.0,00-0035457,34427.0,6063.0,48601.0,3894912.0,32511.0,NaN,2180626.0,JohnTy03,NaN,13615.0,14690.0,NaN,32511.0,865567.0,20838.0,865567.0,Tyron Johnson,tyron johnson,WR,FA,1996-01-08,28.1,2019.0,NaN,NaN,NaN,NaN,73.0,193.0,Oklahoma State,2025,tyron johnson,tyron johnson_32511,32511,johnson
2679,16859,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2180626.0,NaN,NaN,NaN,NaN,NaN,32511.0,0.0,NaN,NaN,Tyron Billy-Johnson,tyron billy-johnson,XX,FA,1996-01-08,29.1,2019.0,NaN,NaN,NaN,NaN,73.0,193.0,Oklahoma State,2025,tyron billy-johnson,tyron billy-johnson_32511,32511,billy-johnson


In [91]:
yahoo_players[yahoo_players['player_id'] == 32511]

,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season,cleaned_name,match_key,last_name
7031,8,WR,nfl.p.32511,Dal,Dallas Cowboys,nfl.t.6,https://sports.yahoo.com/nfl/teams/dallas/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/bdGameuMdoTd...,https://s.yimg.com/iu/api/res/1.2/bdGameuMdoTd...,False,False,False,0,Tyron,Tyron,Tyron Billy-Johnson,Billy-Johnson,32511,390.p.32511,O,WR,https://sports.yahoo.com/nfl/players/32511,2019,tyron billy-johnson,tyron billy-johnson_32511,billy-johnson
7953,10,WR,nfl.p.32511,Dal,Dallas Cowboys,nfl.t.6,https://sports.yahoo.com/nfl/teams/dallas/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/bdGameuMdoTd...,https://s.yimg.com/iu/api/res/1.2/bdGameuMdoTd...,False,False,False,0,Tyron,Tyron,Tyron Billy-Johnson,Billy-Johnson,32511,399.p.32511,O,WR,https://sports.yahoo.com/nfl/players/32511,2020,tyron billy-johnson,tyron billy-johnson_32511,billy-johnson
8953,7,WR,nfl.p.32511,Dal,Dallas Cowboys,nfl.t.6,https://sports.yahoo.com/nfl/teams/dallas/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/bdGameuMdoTd...,https://s.yimg.com/iu/api/res/1.2/bdGameuMdoTd...,False,False,False,0,Tyron,Tyron,Tyron Billy-Johnson,Billy-Johnson,32511,406.p.32511,O,WR,https://sports.yahoo.com/nfl/players/32511,2021,tyron billy-johnson,tyron billy-johnson_32511,billy-johnson
9946,9,WR,nfl.p.32511,Dal,Dallas Cowboys,nfl.t.6,https://sports.yahoo.com/nfl/teams/dallas/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/bdGameuMdoTd...,https://s.yimg.com/iu/api/res/1.2/bdGameuMdoTd...,False,False,False,0,Tyron,Tyron,Tyron Billy-Johnson,Billy-Johnson,32511,414.p.32511,O,WR,https://sports.yahoo.com/nfl/players/32511,2022,tyron billy-johnson,tyron billy-johnson_32511,billy-johnson
10982,7,WR,nfl.p.32511,Dal,Dallas Cowboys,nfl.t.6,https://sports.yahoo.com/nfl/teams/dallas/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/bdGameuMdoTd...,https://s.yimg.com/iu/api/res/1.2/bdGameuMdoTd...,False,False,False,0,Tyron,Tyron,Tyron Billy-Johnson,Billy-Johnson,32511,423.p.32511,O,WR,https://sports.yahoo.com/nfl/players/32511,2023,tyron billy-johnson,tyron billy-johnson_32511,billy-johnson
12038,7,WR,nfl.p.32511,Dal,Dallas Cowboys,nfl.t.6,https://sports.yahoo.com/nfl/teams/dallas/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/bdGameuMdoTd...,https://s.yimg.com/iu/api/res/1.2/bdGameuMdoTd...,False,False,False,0,Tyron,Tyron,Tyron Billy-Johnson,Billy-Johnson,32511,449.p.32511,O,WR,https://sports.yahoo.com/nfl/players/32511,2024,tyron billy-johnson,tyron billy-johnson_32511,billy-johnson


In [92]:
master_raw = master_raw.drop_duplicates(subset='mapped_ids', keep='first')

In [93]:
master_raw[master_raw['cleaned_name'] == "travis hunter"]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids,last_name
44,17074,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025,travis hunter,travis hunter_99001,41787,hunter


In [94]:
master_raw.loc[master_raw['mapped_ids'] == 32511, ['cleaned_name', 'match_key', 'last_name']] = ['tyron billy-johnson','tyron billy-johnson_32511', 'billy-johnson']

In [95]:
master_raw[master_raw['mapped_ids'] == 32511]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids,last_name
2297,14117,064c4eda-1b10-40ac-a9d2-66caf76a213a,18804.0,00-0035457,34427.0,6063.0,48601.0,3894912.0,32511.0,NaN,2180626.0,JohnTy03,NaN,13615.0,14690.0,NaN,32511.0,865567.0,20838.0,865567.0,Tyron Johnson,tyron johnson,WR,FA,1996-01-08,28.1,2019.0,NaN,NaN,NaN,NaN,73.0,193.0,Oklahoma State,2025,tyron billy-johnson,tyron billy-johnson_32511,32511,billy-johnson


In [96]:
master_raw[['stats_id', 'mapped_ids', 'match_key']].nunique()

stats_id      4329
mapped_ids    4329
match_key     4329
dtype: int64

In [97]:
master_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4329 entries, 0 to 12040
Data columns (total 39 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   mfl_id            4329 non-null   int64  
 1   sportradar_id     3253 non-null   object 
 2   fantasypros_id    2487 non-null   float64
 3   gsis_id           3328 non-null   object 
 4   pff_id            3135 non-null   float64
 5   sleeper_id        2833 non-null   float64
 6   nfl_id            3080 non-null   float64
 7   espn_id           3444 non-null   float64
 8   yahoo_id          2679 non-null   float64
 9   fleaflicker_id    489 non-null    float64
 10  cbs_id            4076 non-null   float64
 11  pfr_id            3520 non-null   object 
 12  cfbref_id         1830 non-null   object 
 13  rotowire_id       4220 non-null   float64
 14  rotoworld_id      3179 non-null   float64
 15  ktc_id            394 non-null    float64
 16  stats_id          4329 non-null   float64
 17 

In [98]:
yahoo_players['match_key'].nunique()

4263

In [101]:
master_raw.loc[master_raw['stats_id'] == 41787, 'mapped_ids'] = 99001

In [103]:
master_raw[~(master_raw['mapped_ids'].isin(yahoo_players['player_id']))]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids,last_name
358,16590,ec7b8cf0-0bb9-11ef-bb45-99966e93bb71,23002.0,NaN,NaN,12259.0,NaN,NaN,NaN,NaN,3123935.0,NaN,NaN,16999.0,NaN,NaN,41559.0,0.0,NaN,NaN,Taulia Tagovailoa,taulia tagovailoa,QB,FA,2000-02-15,25.7,2024.0,NaN,NaN,NaN,NaN,71.0,200.0,Maryland,2025,taulia tagovailoa,taulia tagovailoa_41559,41559,tagovailoa
585,16818,8e6fc170-07d2-11ef-95d0-7db2ea91136a,NaN,NaN,NaN,12188.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41477.0,0.0,NaN,NaN,Justin Strong,justin strong,RB,FA,NaN,NaN,2024.0,NaN,NaN,NaN,NaN,72.0,205.0,Southern Illinois,2025,justin strong,justin strong_41477,41477,strong
879,16335,9cedc261-40ff-4fe3-80e1-e128dcceede0,25567.0,00-0038614,82482.0,10994.0,56065.0,4373684.0,40225.0,NaN,2969142.0,MatlSc00,scott-matlock-1,16767.0,NaN,NaN,40225.0,0.0,24457.0,1130595.0,Scott Matlock,scott matlock,RB,LAC,2000-06-28,25.3,2023.0,6.0,23.0,200.0,NaN,76.0,296.0,Boise State,2025,scott matlock,scott matlock_40225,40225,matlock
938,16394,53bcdfe0-ec71-11ed-8bf2-e7fff7735d4a,NaN,NaN,NaN,11344.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17041.0,NaN,NaN,40592.0,0.0,NaN,974075.0,Keytaon Thompson,keytaon thompson,WR,FA,1998-10-23,25.3,2023.0,NaN,NaN,NaN,NaN,77.0,218.0,Virginia,2025,keytaon thompson,keytaon thompson_40592,40592,thompson
3285,13282,dd50e8ab-518b-44a1-9c71-2bc2af74ee7c,NaN,00-0033552,11856.0,4130.0,44913.0,3043144.0,30214.0,NaN,2061118.0,LangBr00,NaN,12022.0,12345.0,NaN,30214.0,734330.0,18976.0,NaN,Brendan Langley,brendan langley,WR,FA,1994-10-16,26.6,2017.0,3.0,37.0,101.0,NaN,72.0,199.0,Lamar,2025,brendan langley,brendan langley_30214,30214,langley
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12034,9793,20af511b-03b7-4905-bc41-158f286b99c6,NaN,NaN,NaN,2864.0,2553937.0,12899.0,29138.0,NaN,2175732.0,NaN,NaN,6002.0,5243.0,NaN,9736.0,245814.0,17332.0,NaN,Mike Reilly,mike reilly,QB,STL,1985-01-25,36.3,0.0,NaN,NaN,NaN,NaN,76.0,240.0,Central Washington,2025,mike reilly,mike reilly_9736,9736,reilly
12036,9796,NaN,NaN,00-0026792,NaN,NaN,NaN,13027.0,NaN,NaN,1675238.0,WillPa01,NaN,6401.0,5595.0,NaN,9721.0,267416.0,NaN,NaN,Patrick Williams,patrick williams,WR,BAL,1986-01-13,35.3,0.0,NaN,NaN,NaN,NaN,73.0,204.0,Colorado,2025,patrick williams,patrick williams_9721,9721,williams
12038,9800,NaN,NaN,00-0024825,NaN,NaN,NaN,11142.0,NaN,NaN,1226291.0,NordJa00,NaN,5467.0,4608.0,NaN,8692.0,219960.0,NaN,NaN,Jake Nordin,jake nordin,TE,DET,1984-07-08,36.8,0.0,NaN,NaN,NaN,NaN,75.0,255.0,Northern Illinois,2025,jake nordin,jake nordin_8692,8692,nordin
12039,9806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6202.0,5498.0,NaN,9122.0,463275.0,NaN,NaN,Sam Giguere,sam giguere,WR,FA,1985-07-11,35.8,0.0,NaN,NaN,NaN,NaN,71.0,215.0,Sherbrooke,2025,sam giguere,sam giguere_9122,9122,giguere


In [104]:
yahoo_players[yahoo_players['player_id'] == 41787]

,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season,cleaned_name,match_key,last_name


In [105]:
merged_ids = master_raw.merge(yahoo_players, left_on='mapped_ids', right_on='player_id', how='outer', suffixes=['_master', '_yahoo'])
merged_ids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15412 entries, 0 to 15411
Data columns (total 67 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   mfl_id                    13355 non-null  float64
 1   sportradar_id             12254 non-null  object 
 2   fantasypros_id            10953 non-null  float64
 3   gsis_id                   12293 non-null  object 
 4   pff_id                    12057 non-null  float64
 5   sleeper_id                11819 non-null  float64
 6   nfl_id                    11960 non-null  float64
 7   espn_id                   12394 non-null  float64
 8   yahoo_id                  11604 non-null  float64
 9   fleaflicker_id            1252 non-null   float64
 10  cbs_id                    13082 non-null  float64
 11  pfr_id                    12384 non-null  object 
 12  cfbref_id                 6826 non-null   object 
 13  rotowire_id               13246 non-null  float64
 14  rotowo

In [107]:
display(merged_ids[(merged_ids['gsis_id'].isna()) & (merged_ids['primary_position'] != 'DEF')])
merged_ids[(merged_ids['gsis_id'].isna()) & (merged_ids['primary_position'] != 'DEF')].tail()

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name_master,match_key_master,mapped_ids,last_name_master,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season,cleaned_name_yahoo,match_key_yahoo,last_name_yahoo
7,2785.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500.0,DoerCh00,chris-doering-1,1223.0,700.0,NaN,3731.0,0.0,NaN,NaN,Chris Doering,chris doering,WR,FA,1973-05-19,48.0,1996.0,6.0,18.0,185.0,NaN,76.0,201.0,Florida,2025.0,chris doering,chris doering_3731,3731.0,doering,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3161.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,383.0,NaN,NaN,3807.0,0.0,NaN,NaN,Harold Morrow,harold morrow,RB,FA,1973-02-24,48.2,0.0,NaN,NaN,NaN,NaN,NaN,232.0,Auburn,2025.0,harold morrow,harold morrow_3807,3807.0,morrow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3185.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1453.0,NaN,NaN,NaN,801.0,NaN,3853.0,0.0,NaN,NaN,Chris Hetherington,chris hetherington,RB,FA,1972-11-27,48.4,0.0,NaN,NaN,NaN,NaN,75.0,253.0,Yale,2025.0,chris hetherington,chris hetherington_3853,3853.0,hetherington,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,3250.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1624.0,NaN,NaN,286.0,2604.0,NaN,3873.0,0.0,NaN,NaN,Kelly Holcomb,kelly holcomb,QB,FA,1973-07-09,47.8,0.0,NaN,NaN,NaN,NaN,74.0,212.0,Middle Tennessee Sta,2025.0,kelly holcomb,kelly holcomb_3873,3873.0,holcomb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,2917.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DunnWa00,warrick-dunn-1,738.0,2117.0,NaN,3893.0,0.0,NaN,NaN,Warrick Dunn,warrick dunn,RB,FA*,1975-01-05,46.3,1997.0,1.0,12.0,12.0,WarrickDunn,69.0,180.0,Florida State,2025.0,warrick dunn,warrick dunn_3893,3893.0,dunn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,RB,nfl.p.42587,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['RB'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0.0,Nay'Quan,Nay'Quan,Nay'Quan Wright,Wright,42587.0,461.p.42587,O,RB,https://sports.yahoo.com/nfl/players/42587,2025.0,nayquan wright,nayquan wright_42587,wright
15022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,TE,nfl.p.42592,Det,Detroit Lions,nfl.t.8,https://sports.yahoo.com/nfl/teams/detroit/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0.0,Gunnar,Gunnar,Gunnar Oakes,Oakes,42592.0,461.p.42592,O,TE,https://sports.yahoo.com/nfl/players/42592,2025.0,gunnar oakes,gunn

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name_master,match_key_master,mapped_ids,last_name_master,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season,cleaned_name_yahoo,match_key_yahoo,last_name_yahoo
15021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,RB,nfl.p.42587,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['RB'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0.0,Nay'Quan,Nay'Quan,Nay'Quan Wright,Wright,42587.0,461.p.42587,O,RB,https://sports.yahoo.com/nfl/players/42587,2025.0,nayquan wright,nayquan wright_42587,wright
15022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,TE,nfl.p.42592,Det,Detroit Lions,nfl.t.8,https://sports.yahoo.com/nfl/teams/detroit/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0.0,Gunnar,Gunnar,Gunnar Oakes,Oakes,42592.0,461.p.42592,O,TE,https://sports.yahoo.com/nfl/players/42592,2025.0,gunnar oakes,gunnar oakes_42592,oakes
15023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,RB,nfl.p.42596,NE,New England Patriots,nfl.t.17,https://sports.yahoo.com/nfl/teams/new-england/,['RB'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0.0,Shane,Shane,Shane Watts,Watts,42596.0,461.p.42596,O,RB,https://sports.yahoo.com/nfl/players/42596,2025.0,shane watts,shane watts_42596,watts
15024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,WR,nfl.p.42597,NO,New Orleans Saints,nfl.t.18,https://sports.yahoo.com/nfl/teams/new-orleans/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0.0,Roderick,Roderick,Roderick Daniels Jr.,Daniels Jr.,42597.0,461.p.42597,O,WR,https://sports.yahoo.com/nfl/players/42597,2025.0,roderick daniels,roderick daniels_42597,daniels
15026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,WR,nfl.p.42602,NE,New England Patriots,nfl.t.17,https://sports.yahoo.com/nfl/teams/new-england/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0.0,Phil,Phil,Phil Lutz,Lutz,42602.0,461.p.42602,O,WR,https://sports.yahoo.com/nfl/players/42602,2025.0,phil lutz,phil lutz_42602,lutz


In [108]:
merged_ids = merged_ids.dropna(subset='player_id')
display(merged_ids[(merged_ids['gsis_id'].isna()) & (merged_ids['primary_position'] != 'DEF')])

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name_master,match_key_master,mapped_ids,last_name_master,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season,cleaned_name_yahoo,match_key_yahoo,last_name_yahoo
1205,8641.0,f4f9983e-2334-4c10-afac-449ea4e83a90,NaN,NaN,NaN,504.0,2506746.0,13174.0,NaN,4557.0,1114549.0,NaN,NaN,5061.0,4145.0,NaN,8054.0,156008.0,10412.0,NaN,Micheal Spurlock,micheal spurlock,WR,FA,1983-01-31,38.3,2006.0,NaN,NaN,NaN,NaN,71.0,210.0,Mississippi,2025.0,micheal spurlock,micheal spurlock_8054,8054.0,spurlock,9.0,WR,nfl.p.8054,Chi,Chicago Bears,nfl.t.3,https://sports.yahoo.com/nfl/teams/chicago/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/pCFEkFDnlnV8...,https://s.yimg.com/iu/api/res/1.2/pCFEkFDnlnV8...,False,False,False,0.0,Micheal,Micheal,Micheal Spurlock,Spurlock,8054.0,331.p.8054,O,WR,https://sports.yahoo.com/nfl/players/8054,2014.0,micheal spurlock,micheal spurlock_8054,spurlock
2065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,WR,nfl.p.9668,Bal,Baltimore Ravens,nfl.t.33,https://sports.yahoo.com/nfl/teams/baltimore/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0.0,Dobson,Dobson,Dobson Collins,Collins,9668.0,359.p.9668,O,WR,https://sports.yahoo.com/nfl/players/9668,2016.0,dobson collins,dobson collins_9668,collins
2475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,WR,nfl.p.24746,Car,Carolina Panthers,nfl.t.29,https://sports.yahoo.com/nfl/teams/carolina/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/M8gJ9e0k7poq...,https://s.yimg.com/iu/api/res/1.2/M8gJ9e0k7poq...,False,False,False,0.0,Teddy,Teddy,Teddy Williams,Williams,24746.0,331.p.24746,O,WR,https://sports.yahoo.com/nfl/players/24746,2014.0,teddy williams,teddy williams_24746,williams
2665,10359.0,26dd8e50-8f7a-43d6-8a98-addaaeb8194b,9953.0,NaN,NaN,804.0,2495284.0,14209.0,24898.0,NaN,1695497.0,GateCl00,NaN,7383.0,6583.0,NaN,24898.0,557952.0,12754.0,NaN,Clyde Gates,clyde gates,WR,TEN,1986-06-13,34.9,2011.0,4.0,14.0,111.0,EdmondGates,72.0,197.0,Abilene Christian,2025.0,clyde gates,clyde gates_24898,24898.0,gates,11.0,WR,nfl.p.24898,NYJ,New York Jets,nfl.t.20,https://sports.yahoo.com/nfl/teams/ny-jets/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/s.ugCuaAFaJD...,https://s.yimg.com/iu/api/res/1.2/s.ugCuaAFaJD...,False,False,False,0.0,Edmond,Edmond,Edmond Gates,Gates,24898.0,331.p.24898,O,WR,https://sports.yahoo.com/nfl/players/24898,2014.0,edmond gates,edmond gates_24898,gates
2666,10359.0,26dd8e50-8f7a-43d6-8a98-addaaeb8194b,9953.0,NaN,NaN,804.0,2495284.0,14209.0,24898.0,NaN,1695497.0,GateCl00,NaN,7383.0,6583.0,NaN,24898.0,557952.0,12754.0,NaN,Clyde Gates,clyde gates,WR,TEN,1986-06-13,34.9,2011.0,4.0,14.0,111.0,EdmondGates,72.0,197.0,Abilene Christian,2025.0,clyde gates,clyde gates_24898,24898.0,gates,5.0,WR,nfl.p.24898,NYJ,New York Jets,nfl.t.20,https://sports.yahoo.com/nfl/teams/ny-jets/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/s.ugCuaAFaJD...,https://s.yimg.com/iu/api/res/1.2/s.ugCuaAFaJD...,False,False,False,0.0,Edmond,Edmond,Edmond Gates,Gates,24898.0,348.p.24898,O,WR,https://sports.yahoo.com/nfl/players/24898,2015.0,edmond

In [110]:
merged_ids[(merged_ids['cleaned_name_master'] == 'travis hunter') | (merged_ids['cleaned_name_yahoo'] == 'travis hunter')]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name_master,match_key_master,mapped_ids,last_name_master,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season,cleaned_name_yahoo,match_key_yahoo,last_name_yahoo
15027,17074.0,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025.0,travis hunter,travis hunter_99001,99001.0,hunter,8.0,WR,nfl.p.41787,Jax,Jacksonville Jaguars,nfl.t.30,https://sports.yahoo.com/nfl/teams/jacksonville/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/VMwr0OLB8DWK...,https://s.yimg.com/iu/api/res/1.2/VMwr0OLB8DWK...,False,False,False,0.0,Travis,Travis,Travis Hunter,Hunter,99001.0,461.p.99001,O,WR,https://sports.yahoo.com/nfl/players/41787,2025.0,travis hunter,travis hunter_99001,hunter


In [111]:
merged_ids.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14096 entries, 0 to 15411
Data columns (total 67 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   mfl_id                    12039 non-null  float64
 1   sportradar_id             11974 non-null  object 
 2   fantasypros_id            10879 non-null  float64
 3   gsis_id                   11776 non-null  object 
 4   pff_id                    11643 non-null  float64
 5   sleeper_id                11783 non-null  float64
 6   nfl_id                    11576 non-null  float64
 7   espn_id                   11810 non-null  float64
 8   yahoo_id                  11522 non-null  float64
 9   fleaflicker_id            1016 non-null   float64
 10  cbs_id                    11956 non-null  float64
 11  pfr_id                    11547 non-null  object 
 12  cfbref_id                 6282 non-null   object 
 13  rotowire_id               12028 non-null  float64
 14  rotoworld_i

In [113]:
merged_ids.loc[merged_ids['stats_id'].isna(), 'stats_id'] = merged_ids.loc[merged_ids['stats_id'].isna(), 'player_id']

In [114]:
merged_ids.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14096 entries, 0 to 15411
Data columns (total 67 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   mfl_id                    12039 non-null  float64
 1   sportradar_id             11974 non-null  object 
 2   fantasypros_id            10879 non-null  float64
 3   gsis_id                   11776 non-null  object 
 4   pff_id                    11643 non-null  float64
 5   sleeper_id                11783 non-null  float64
 6   nfl_id                    11576 non-null  float64
 7   espn_id                   11810 non-null  float64
 8   yahoo_id                  11522 non-null  float64
 9   fleaflicker_id            1016 non-null   float64
 10  cbs_id                    11956 non-null  float64
 11  pfr_id                    11547 non-null  object 
 12  cfbref_id                 6282 non-null   object 
 13  rotowire_id               12028 non-null  float64
 14  rotoworld_i

In [115]:
merged_ids[merged_ids['stats_id'] != merged_ids['mapped_ids']]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name_master,match_key_master,mapped_ids,last_name_master,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season,cleaned_name_yahoo,match_key_yahoo,last_name_yahoo
2065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9668.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,WR,nfl.p.9668,Bal,Baltimore Ravens,nfl.t.33,https://sports.yahoo.com/nfl/teams/baltimore/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0.0,Dobson,Dobson,Dobson Collins,Collins,9668.0,359.p.9668,O,WR,https://sports.yahoo.com/nfl/players/9668,2016.0,dobson collins,dobson collins_9668,collins
2475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24746.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,WR,nfl.p.24746,Car,Carolina Panthers,nfl.t.29,https://sports.yahoo.com/nfl/teams/carolina/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/M8gJ9e0k7poq...,https://s.yimg.com/iu/api/res/1.2/M8gJ9e0k7poq...,False,False,False,0.0,Teddy,Teddy,Teddy Williams,Williams,24746.0,331.p.24746,O,WR,https://sports.yahoo.com/nfl/players/24746,2014.0,teddy williams,teddy williams_24746,williams
2834,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25050.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,TE,nfl.p.25050,Cle,Cleveland Browns,nfl.t.5,https://sports.yahoo.com/nfl/teams/cleveland/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/SKdKxY1l7uWA...,https://s.yimg.com/iu/api/res/1.2/SKdKxY1l7uWA...,False,False,False,0.0,Martell,Martell,Martell Webb,Webb,25050.0,331.p.25050,O,TE,https://sports.yahoo.com/nfl/players/25050,2014.0,martell webb,martell webb_25050,webb
2867,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25123.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,"WR,TE",nfl.p.25123,Mia,Miami Dolphins,nfl.t.15,https://sports.yahoo.com/nfl/teams/miami/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0.0,Raymond,Raymond,Raymond Webber,Webber,25123.0,331.p.25123,O,WR,https://sports.yahoo.com/nfl/players/25123,2014.0,raymond webber,raymond webber_25123,webber
2920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25277.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,WR,nfl.p.25277,Chi,Chicago Bears,nfl.t.3,https://sports.yahoo.com/nfl/teams/chicago/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/ErQQSpQt6mru...,https://s.yimg.com/iu/api/res/1.2/ErQQSpQt6mru...,False,False,False,0.0,John,John,John Chiles,Chiles,25277.0,348.p.25277,O,WR,https://sports.yahoo.com/nfl/players/25277,2015.0,john chiles,john chiles_25277,chiles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100034.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,DEF,nfl.p.100

In [116]:
merged_ids.loc[merged_ids['mapped_ids'].isna(), 'mapped_ids'] = merged_ids.loc[merged_ids['mapped_ids'].isna(), 'stats_id']

In [117]:
merged_ids[merged_ids['stats_id'] != merged_ids['mapped_ids']]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name_master,match_key_master,mapped_ids,last_name_master,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season,cleaned_name_yahoo,match_key_yahoo,last_name_yahoo
15027,17074.0,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025.0,travis hunter,travis hunter_99001,99001.0,hunter,8.0,WR,nfl.p.41787,Jax,Jacksonville Jaguars,nfl.t.30,https://sports.yahoo.com/nfl/teams/jacksonville/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/VMwr0OLB8DWK...,https://s.yimg.com/iu/api/res/1.2/VMwr0OLB8DWK...,False,False,False,0.0,Travis,Travis,Travis Hunter,Hunter,99001.0,461.p.99001,O,WR,https://sports.yahoo.com/nfl/players/41787,2025.0,travis hunter,travis hunter_99001,hunter


In [119]:
merged_ids[merged_ids['last_name_yahoo'] == 'borregales']

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name_master,match_key_master,mapped_ids,last_name_master,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season,cleaned_name_yahoo,match_key_yahoo,last_name_yahoo
12954,15507.0,42a287f6-9a56-4699-8c63-bc54625ff6eb,23403.0,00-0036757,NaN,7961.0,NaN,NaN,33867.0,NaN,2262788.0,NaN,NaN,15617.0,NaN,NaN,33867.0,0.0,NaN,948876.0,Jose Borregales,jose borregales,PK,FA,1997-12-15,25.2,2021.0,NaN,NaN,NaN,NaN,70.0,205.0,Miami (FL),2025.0,jose borregales,jose borregales_33867,33867.0,borregales,9.0,K,nfl.p.33867,TB,Tampa Bay Buccaneers,nfl.t.27,https://sports.yahoo.com/nfl/teams/tampa-bay/,['K'],small,https://s.yimg.com/iu/api/res/1.2/K97wBKJC19ug...,https://s.yimg.com/iu/api/res/1.2/K97wBKJC19ug...,False,False,False,0.0,Jose,Jose,Jose Borregales,Borregales,33867.0,406.p.33867,K,K,https://sports.yahoo.com/nfl/players/33867,2021.0,jose borregales,jose borregales_33867,borregales
12955,15507.0,42a287f6-9a56-4699-8c63-bc54625ff6eb,23403.0,00-0036757,NaN,7961.0,NaN,NaN,33867.0,NaN,2262788.0,NaN,NaN,15617.0,NaN,NaN,33867.0,0.0,NaN,948876.0,Jose Borregales,jose borregales,PK,FA,1997-12-15,25.2,2021.0,NaN,NaN,NaN,NaN,70.0,205.0,Miami (FL),2025.0,jose borregales,jose borregales_33867,33867.0,borregales,11.0,K,nfl.p.33867,TB,Tampa Bay Buccaneers,nfl.t.27,https://sports.yahoo.com/nfl/teams/tampa-bay/,['K'],small,https://s.yimg.com/iu/api/res/1.2/K97wBKJC19ug...,https://s.yimg.com/iu/api/res/1.2/K97wBKJC19ug...,False,False,False,0.0,Jose,Jose,Jose Borregales,Borregales,33867.0,414.p.33867,K,K,https://sports.yahoo.com/nfl/players/33867,2022.0,jose borregales,jose borregales_33867,borregales
14803,17220.0,80dce1ba-c997-47e6-a37e-e92fefcbdc1b,NaN,00-0040200,143934.0,12713.0,58387.0,4569923.0,NaN,NaN,26699665.0,BorrAn00,NaN,18743.0,NaN,NaN,41980.0,0.0,26106.0,NaN,Andres Borregales,andres borregales,PK,NEP,2002-01-02,23.8,2025.0,6.0,6.0,182.0,NaN,71.0,202.0,Miami (FL),2025.0,andres borregales,andres borregales_41980,41980.0,borregales,14.0,K,nfl.p.41980,NE,New England Patriots,nfl.t.17,https://sports.yahoo.com/nfl/teams/new-england/,['K'],small,https://s.yimg.com/iu/api/res/1.2/xgXGV4LH8kz....,https://s.yimg.com/iu/api/res/1.2/xgXGV4LH8kz....,False,False,False,0.0,Andy,Andy,Andy Borregales,Borregales,41980.0,461.p.41980,K,K,https://sports.yahoo.com/nfl/players/41980,2025.0,andy borregales,andy borregales_41980,borregales


In [120]:
merged_ids['player_id'].nunique()

4263

In [121]:
merged_ids.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14096 entries, 0 to 15411
Data columns (total 67 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   mfl_id                    12039 non-null  float64
 1   sportradar_id             11974 non-null  object 
 2   fantasypros_id            10879 non-null  float64
 3   gsis_id                   11776 non-null  object 
 4   pff_id                    11643 non-null  float64
 5   sleeper_id                11783 non-null  float64
 6   nfl_id                    11576 non-null  float64
 7   espn_id                   11810 non-null  float64
 8   yahoo_id                  11522 non-null  float64
 9   fleaflicker_id            1016 non-null   float64
 10  cbs_id                    11956 non-null  float64
 11  pfr_id                    11547 non-null  object 
 12  cfbref_id                 6282 non-null   object 
 13  rotowire_id               12028 non-null  float64
 14  rotoworld_i

In [124]:
merged_ids = merged_ids.loc[(merged_ids['gsis_id'].notna()) | (merged_ids['primary_position'] == 'DEF')]

In [125]:
merged_ids.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12160 entries, 0 to 15411
Data columns (total 67 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   mfl_id                    11776 non-null  float64
 1   sportradar_id             11766 non-null  object 
 2   fantasypros_id            10803 non-null  float64
 3   gsis_id                   11776 non-null  object 
 4   pff_id                    11643 non-null  float64
 5   sleeper_id                11605 non-null  float64
 6   nfl_id                    11539 non-null  float64
 7   espn_id                   11678 non-null  float64
 8   yahoo_id                  11396 non-null  float64
 9   fleaflicker_id            1015 non-null   float64
 10  cbs_id                    11720 non-null  float64
 11  pfr_id                    11531 non-null  object 
 12  cfbref_id                 6271 non-null   object 
 13  rotowire_id               11774 non-null  float64
 14  rotoworld_i

In [127]:
merged_ids[['stats_id', 'player_id', 'gsis_id']].nunique()

stats_id     2843
player_id    2843
gsis_id      2810
dtype: int64

In [ ]:
merg

In [135]:
merged_ids[merged_ids['gsis_id'].isna()]['primary_position'].value_counts()

primary_position
DEF    384
Name: count, dtype: int64

In [136]:
merged_ids.loc[merged_ids['gsis_id'].isna(), 'gsis_id'] = merged_ids.loc[merged_ids['gsis_id'].isna(), 'name.full']

In [137]:
merged_ids[['stats_id', 'player_id', 'gsis_id']].nunique()

stats_id     2843
player_id    2843
gsis_id      2842
dtype: int64

In [146]:
# Find gsis_ids that map to multiple different stats_ids
multi_stats = merged_ids.groupby('gsis_id')['stats_id'].nunique()
multi_stats[multi_stats > 1]

gsis_id
00-0031320    2
Name: stats_id, dtype: int64

In [149]:
merged_ids = merged_ids.loc[merged_ids['mapped_ids'] != 27502]

In [151]:
merged_ids[['stats_id', 'player_id', 'gsis_id', 'mapped_ids']].nunique()

stats_id      2842
player_id     2842
gsis_id       2842
mapped_ids    2842
dtype: int64

In [152]:
merged_ids.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12157 entries, 0 to 15411
Data columns (total 67 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   mfl_id                    11773 non-null  float64
 1   sportradar_id             11763 non-null  object 
 2   fantasypros_id            10803 non-null  float64
 3   gsis_id                   12157 non-null  object 
 4   pff_id                    11640 non-null  float64
 5   sleeper_id                11602 non-null  float64
 6   nfl_id                    11536 non-null  float64
 7   espn_id                   11675 non-null  float64
 8   yahoo_id                  11393 non-null  float64
 9   fleaflicker_id            1015 non-null   float64
 10  cbs_id                    11717 non-null  float64
 11  pfr_id                    11528 non-null  object 
 12  cfbref_id                 6271 non-null   object 
 13  rotowire_id               11771 non-null  float64
 14  rotoworld_i

In [ ]:
# master_raw[master_raw['gsis_id'].isna()].head(50)

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids,last_name
55,17085,1edb3290-2518-11f0-972e-efd9d03db8c5,27455.0,NaN,NaN,12972.0,NaN,4431597.0,NaN,NaN,26733219.0,NaN,NaN,18651.0,NaN,NaN,42279.0,0.0,NaN,NaN,Roc Taylor,roc taylor,WR,FA,2002-08-26,23.1,2025.0,NaN,NaN,NaN,NaN,73.0,213.0,Memphis,2025,roc taylor,roc taylor_42279,42279,taylor
59,17089,483872b0-2630-11f0-b7f1-df7c830143e7,27218.0,NaN,NaN,12460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18559.0,NaN,NaN,42444.0,0.0,26081.0,NaN,Antwane Wells,antwane wells,WR,NYG,2001-04-02,24.5,2025.0,NaN,NaN,NaN,NaN,72.0,205.0,Ole Miss,2025,antwane wells,antwane wells_42444,42444,wells
60,17090,3e9fe6f0-2619-11f0-8447-731004665991,23063.0,NaN,NaN,13108.0,NaN,4430800.0,NaN,NaN,NaN,NaN,NaN,17067.0,NaN,NaN,42423.0,0.0,NaN,NaN,Julian Fleming,julian fleming,WR,FA,2000-12-19,24.8,2025.0,NaN,NaN,NaN,NaN,74.0,208.0,Penn State,2025,julian fleming,julian fleming_42423,42423,fleming
62,17092,71bd62b0-3ccb-11f0-b22b-71ee09ef21d6,27450.0,NaN,NaN,13225.0,NaN,4587977.0,NaN,NaN,29052184.0,NaN,NaN,18468.0,NaN,NaN,42545.0,0.0,NaN,NaN,Sam Brown,sam brown,WR,FA,2002-08-15,23.2,2025.0,NaN,NaN,NaN,NaN,74.0,200.0,Miami (FL),2025,sam brown,sam brown_42545,42545,brown
63,17093,8a410ba0-23a5-11f0-8d5d-1b4f09989b68,26061.0,NaN,NaN,12810.0,NaN,4429184.0,NaN,NaN,3163150.0,NaN,NaN,18570.0,NaN,NaN,42111.0,0.0,NaN,NaN,Elijhah Badger,elijhah badger,WR,FA,2001-08-15,24.2,2025.0,NaN,NaN,NaN,NaN,73.0,192.0,Florida,2025,elijhah badger,elijhah badger_42111,42111,badger
65,17095,a6f8abf0-25f7-11f0-8ecb-77a06236c534,27075.0,NaN,NaN,12488.0,NaN,4429056.0,NaN,NaN,3161851.0,NaN,NaN,18557.0,NaN,NaN,42404.0,0.0,NaN,NaN,Kobe Hudson,kobe hudson,WR,FA,2001-09-11,24.1,2025.0,NaN,NaN,NaN,NaN,72.0,193.0,UCF,2025,kobe hudson,kobe hudson_42404,42404,hudson
71,17101,557ec8cb-2700-4dcb-8d5e-db0e0e5a8ef5,25413.0,NaN,NaN,12721.0,NaN,4431509.0,NaN,NaN,26694721.0,NaN,NaN,18563.0,NaN,NaN,42020.0,0.0,NaN,NaN,Kaden Prather,kaden prather,WR,FA,2002-09-06,23.1,2025.0,7.0,24.0,NaN,NaN,75.0,204.0,Maryland,2025,kaden prather,kaden prather_42020,42020,prather
235,17266,3557e290-2449-11f0-9b47-b3e02d0eb26f,27649.0,NaN,NaN,12868.0,NaN,4684625.0,NaN,NaN,26699807.0,NaN,NaN,18566.0,NaN,NaN,42173.0,0.0,NaN,NaN,Montrell Johnson,montrell johnson,RB,PHI,2002-10-13,23.0,2025.0,NaN,NaN,NaN,NaN,71.0,212.0,Florida,2025,montrell johnson,montrell johnson_42173,42173,johnson
239,17271,8a118cb0-2551-11f0-9371-f9b494c39630,27293.0,NaN,NaN,12546.0,NaN,4694302.0,NaN,NaN,26728267.0,NaN,NaN,18525.0,NaN,NaN,42345.0,0.0,NaN,NaN,Marcus Yarns,marcus yarns,RB,FA,2001-10-30,24.0,2025.0,NaN,NaN,NaN,NaN,71.0,190.0,Delaware,2025,marcus yarns,marcus yarns_42345,42345,yarns
241,17273,394f23d0-254e-11f0-85e6-ad6d4b9a9b15,25407.0,NaN,NaN,13001.0,NaN,4431618.0,NaN,NaN,26697388.0,NaN,NaN,18926.0,NaN,NaN,42326.0,0.0,NaN,NaN,Mario Williams,mario williams,WR,FA,2003-01-21,22.7,2025.0,NaN,NaN,NaN,NaN,69.0,178.0,Tulane,2025,mario williams,mario williams_42326,42326,williams


None of these players have any stats and seem to be rookies for 2025. This chekcs out as the gsis is the stat for one of the librarie that connects to everything else

In [ ]:
# master_raw = master_raw.dropna(subset='gsis_id')
# master_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3328 entries, 0 to 12040
Data columns (total 39 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   mfl_id            3328 non-null   int64  
 1   sportradar_id     3052 non-null   object 
 2   fantasypros_id    2424 non-null   float64
 3   gsis_id           3328 non-null   object 
 4   pff_id            3135 non-null   float64
 5   sleeper_id        2679 non-null   float64
 6   nfl_id            3053 non-null   float64
 7   espn_id           3268 non-null   float64
 8   yahoo_id          2576 non-null   float64
 9   fleaflicker_id    444 non-null    float64
 10  cbs_id            3286 non-null   float64
 11  pfr_id            3186 non-null   object 
 12  cfbref_id         1538 non-null   object 
 13  rotowire_id       3323 non-null   float64
 14  rotoworld_id      2327 non-null   float64
 15  ktc_id            394 non-null    float64
 16  stats_id          3328 non-null   float64
 17 

In [153]:
merged_ids = merged_ids.dropna(axis=1, how='any')

In [154]:
merged_ids.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12157 entries, 0 to 15411
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   gsis_id                   12157 non-null  object 
 1   stats_id                  12157 non-null  float64
 2   mapped_ids                12157 non-null  float64
 3   bye_weeks.week            12157 non-null  float64
 4   display_position          12157 non-null  object 
 5   editorial_player_key      12157 non-null  object 
 6   editorial_team_abbr       12157 non-null  object 
 7   editorial_team_full_name  12157 non-null  object 
 8   editorial_team_key        12157 non-null  object 
 9   editorial_team_url        12157 non-null  object 
 10  eligible_positions        12157 non-null  object 
 11  headshot.size             12157 non-null  object 
 12  headshot.url              12157 non-null  object 
 13  image_url                 12157 non-null  object 
 14  is_keeper.s

In [ ]:
# merged_ids.to_csv('Raw/player_ids.csv', index=False)